In [27]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import gc
import operator
import networkx as nx

In [3]:
G = nx.watts_strogatz_graph(2000000, 10, 0.5)

In [5]:
assignments = np.concatenate([[k]*10 for k in list(np.random.randint(0, 2, 2000000//10))])

In [6]:
sample = np.random.choice(2000000, 100000)

In [7]:
data = []

In [10]:
data = []
for i in sample:
    neighbor = len(G[i])
    bb_1 = np.sum([assignments[j] for j in G[i]])
    bb_0 = neighbor - bb_1
    bbb_0 = 0
    bbb_1 = 0
    bbb_2 = 0
    bbn_0 = 0
    bbn_1 = 0
    bbn_2 = 0
    open_square_0 = 0
    open_square_1 = 0
    open_square_2 = 0
    open_square_3 = 0
    for j in G[i]:
        for k in G[i]:
            if k > j:
                if np.abs(j-k) <= 5: # this is a simplistic weight to judge if connected (to speed up )
                    if assignments[j] + assignments[k] == 0:
                        bbb_0 += 1
                    elif assignments[j] + assignments[k] == 1:
                        bbb_1 += 1
                    else:
                        bbb_2 += 1
                else:
                    if assignments[j] + assignments[k] == 0:
                        bbn_0 += 1
                    elif assignments[j] + assignments[k] == 1:
                        bbn_1 += 1
                    else:
                        bbn_2 += 1
                    for l in G[i]:
                        if l > k and np.abs(l-k) > 5 and np.abs(l-j) > 5:
                            if assignments[j] + assignments[k] + assignments[l] == 0:
                                open_square_0 += 1
                            elif assignments[j] + assignments[k] + assignments[l]== 1:
                                open_square_1 += 1
                            elif assignments[j] + assignments[k] + assignments[l]== 2:
                                open_square_2 += 1
                            else:
                                open_square_3 += 1
                            
    data.append([i, assignments[i], neighbor, bb_0, bb_1, bbb_0, bbb_1, bbb_2, bbn_0, bbn_1, bbn_2,
                open_square_0, open_square_1, open_square_2, open_square_3])
    if len(data) % 10000 == 0:
        print(len(data))

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [11]:
data = pd.DataFrame.from_records(data)

In [12]:
data.columns = ['id', 'assignment', 'neighbor', 'bb_0', 'bb_1', 'bbb_0', 'bbb_1', 'bbb_2', 'bbn_0', 'bbn_1', 'bbn_2',
                'open_square_0', 'open_square_1', 'open_square_2', 'open_square_3'
               ]

In [13]:
data['bbb_2_normalized'] = data['bbb_2']/(data['bbb_2']+data['bbb_1']+data['bbb_0'])
data['bbb_1_normalized'] = data['bbb_1']/(data['bbb_2']+data['bbb_1']+data['bbb_0'])
data['bbb_0_normalized'] = data['bbb_0']/(data['bbb_2']+data['bbb_1']+data['bbb_0'])

In [14]:
data['bbn_2_normalized'] = data['bbn_2']/(data['bbn_2']+data['bbn_1']+data['bbn_0'])
data['bbn_1_normalized'] = data['bbn_1']/(data['bbn_2']+data['bbn_1']+data['bbn_0'])
data['bbn_0_normalized'] = data['bbn_0']/(data['bbn_2']+data['bbn_1']+data['bbn_0'])

In [15]:
data['open_square_0_normalized'] = data['open_square_0']/(data['open_square_0']+data['open_square_1']+data['open_square_2']+data['open_square_3'])
data['open_square_1_normalized'] = data['open_square_1']/(data['open_square_0']+data['open_square_1']+data['open_square_2']+data['open_square_3'])
data['open_square_2_normalized'] = data['open_square_2']/(data['open_square_0']+data['open_square_1']+data['open_square_2']+data['open_square_3'])
data['open_square_3_normalized'] = data['open_square_3']/(data['open_square_0']+data['open_square_1']+data['open_square_2']+data['open_square_3'])


In [16]:
data['bb_0_normalized'] = data['bb_0']/(data['bb_0']+data['bb_1'])
data['bb_1_normalized'] = data['bb_1']/(data['bb_0']+data['bb_1'])

In [17]:
structural_diversity = []

c = 0

for uid in list(data['id']):
    if c % 10000 == 0:
        print(c)
    structural_diversity.append(
        nx.number_connected_components(nx.subgraph(G, [j for j in nx.neighbors(G, uid) if assignments[j] == 1]))
    )
    c += 1

data['structural_diversity'] = structural_diversity

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [18]:
structural_diversity_1 = []

c = 0

for uid in list(data['id']):
    if c % 10000 == 0:
        print(c)
    structural_diversity_1.append(
        nx.number_connected_components(nx.subgraph(G, [j for j in nx.neighbors(G, uid)]))
    )
    c += 1

data['structural_diversity_1'] = structural_diversity_1

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [19]:
data['gender'] = np.random.randint(0, 2, len(data))

In [21]:
probabilities = []

for replicate in range(100):
    probabilities_mc = []
    assignments = np.concatenate([[k]*10 for k in list(np.random.randint(0, 2, 2000000//10))])
    r = np.random.randint(10)
    assignments = np.concatenate([assignments[r:], assignments[:r]])
    print(replicate)
    for i in sample:
        neighbor = len(G[i])
        bb_1 = np.sum([assignments[j] for j in G[i]])
        bb_0 = neighbor - bb_1
        bbb_0 = 0
        bbb_1 = 0
        bbb_2 = 0
        bbn_0 = 0
        bbn_1 = 0
        bbn_2 = 0
        open_square_0 = 0
        open_square_1 = 0
        open_square_2 = 0
        open_square_3 = 0

        for j in G[i]:
            for k in G[i]:
                if k > j:
                    if np.abs(j-k) <= 5:
                        if assignments[j] + assignments[k] == 0:
                            bbb_0 += 1
                        elif assignments[j] + assignments[k] == 1:
                            bbb_1 += 1
                        else:
                            bbb_2 += 1
                    else:
                        if assignments[j] + assignments[k] == 0:
                            bbn_0 += 1
                        elif assignments[j] + assignments[k] == 1:
                            bbn_1 += 1
                        else:
                            bbn_2 += 1
                        for l in G[i]:
                            if l > k and np.abs(l-k) > 5 and np.abs(l-j) > 5:
                                if assignments[j] + assignments[k] + assignments[l] == 0:
                                    open_square_0 += 1
                                elif assignments[j] + assignments[k] + assignments[l]== 1:
                                    open_square_1 += 1
                                elif assignments[j] + assignments[k] + assignments[l]== 2:
                                    open_square_2 += 1
                                else:
                                    open_square_3 += 1

        probabilities_mc.append([bb_0, bb_1, bbb_0, bbb_1, bbb_2, bbn_0, bbn_1, bbn_2,
                                 open_square_0, open_square_1, open_square_2, open_square_3, assignments[i]
                                ])
        if len(probabilities_mc) % 10000 == 0:
            print(len(probabilities_mc))
    probabilities.append(probabilities_mc)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
1
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
2
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
3
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
4
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
5
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
6
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
7
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
8
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
9
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
10
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
11
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
12
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
13
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
14
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
15
10000
20000
30000
40000
50000
60000
70000
80000

In [22]:
probabilities = np.array(probabilities).T

In [28]:
data.to_csv('data_ws.csv')
probabilities = np.save('probabilities_ws.npy', probabilities)

# #
#saved to g drive: https://drive.google.com/drive/folders/1PuPXJLVqv_i2sBYtD08a5RhSaxmRhwG4 


In [2]:
# # can download it to either local or upload to bento and skip all the previous commands
data = pd.read_csv('data.csv')
probabilities = np.load('probabilities_ws.npy', allow_pickle=True)

In [183]:
new_probabilities = {}

for assign in [0, 1]:
    new_probabilities['bbb_2_normalized'] =  probabilities[4]/(probabilities[4]+probabilities[3]+probabilities[2])
    new_probabilities['bbb_1_normalized'] =  probabilities[3]/(probabilities[4]+probabilities[3]+probabilities[2])
    new_probabilities['bbb_0_normalized'] =  probabilities[2]/(probabilities[4]+probabilities[3]+probabilities[2])

    new_probabilities['bbn_2_normalized'] =  probabilities[7]/(probabilities[7]+probabilities[6]+probabilities[5])
    new_probabilities['bbn_1_normalized'] =  probabilities[6]/(probabilities[7]+probabilities[6]+probabilities[5])
    new_probabilities['bbn_0_normalized'] =  probabilities[5]/(probabilities[7]+probabilities[6]+probabilities[5])

    new_probabilities['bb_1_normalized'] =  probabilities[1]/(probabilities[1]+probabilities[0])
    new_probabilities['bb_0_normalized'] =  probabilities[0]/(probabilities[1]+probabilities[0])

    new_probabilities['open_square_0_normalized'] =  probabilities[8]/(probabilities[8]+probabilities[9]+probabilities[10]+probabilities[11])
    new_probabilities['open_square_1_normalized'] =  probabilities[9]/(probabilities[8]+probabilities[9]+probabilities[10]+probabilities[11])
    new_probabilities['open_square_2_normalized'] =  probabilities[10]/(probabilities[8]+probabilities[9]+probabilities[10]+probabilities[11])
    new_probabilities['open_square_3_normalized'] =  probabilities[11]/(probabilities[8]+probabilities[9]+probabilities[10]+probabilities[11])
    
    new_probabilities['assignment']  = probabilities[-1]



/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/Users/yuany/Library/Python/3.

In [335]:
class causalPartition:
    result_separate = None
    result_eht = None
    
    def __init__(self, df, probabilities, treatment, ratio_train=0.5):
        """
        Ratio train is used to split the sample in the beginning to help construct honest estimator;
        By default it is 50% vs 50% to make the training and estimation sets have the roguhly same widths for CIs
        """
        self.df = df
        self.probabilities = probabilities
        self.treatment = treatment
        self.idx_tr = np.random.random(len(df)) < ratio_train
        self.idx_est = np.logical_not(self.idx_tr)
        self.df_train = df[self.idx_tr]
        self.df_est = df[np.logical_not(self.idx_tr)]
        self.result_separate = {1: None, 0: None}


    def _contain_zero(self, probabilities, rules, eps, delta, treated):
        """
        For each observation (indicated by an element in the vector),
        whether it has <= eps probability to belong to the partition implied by [rules]
        """
        if treated is None:
            # Given 
            return np.mean(np.product([probabilities[key] <= th for key, sign, th in rules if sign == 0] + \
                            [probabilities[key] > th for key, sign, th in rules if sign == 1],
                           axis=0) > 0, axis=1
                          ) <= eps

        else:
            # Given also consider the treated conditions for egos. 
            # In separate trees, the treatment conditions for egos should also be considered
            return np.mean(np.product([probabilities[key] <= th for key, sign, th in rules if sign == 0] + \
                            [probabilities[key] > th for key, sign, th in rules if sign == 1] + \
                             probabilities[self.treatment] == treated,
                           axis=0) > 0, axis=1
                          ) <= eps
    
    
    def _hajek_se(self, d, p, outcome):
        """
        The taylor linearization for hajek se 
        (I use WLS directly for non-separate cases; but in that case se is usually even larger)
        """
        average_hajek_var_up = np.sum( ((d[outcome]/p) ** 2) * (1 - p) )
        average_hajek_var_down = np.sum( ((1.0/p) ** 2) * (1 - p) )
        average_hajek_cov = np.sum( ((1.0/p) ** 2) * d[outcome] * (1 - p) )
        average_hajek_sum_up = np.sum(d[outcome]/p)
        average_hajek_sum_down = np.sum(1.0/p)

        se = np.sqrt(1.0 / (average_hajek_sum_down**2) * average_hajek_var_up + \
                (average_hajek_sum_up**2) / (average_hajek_sum_down**4) * average_hajek_var_down + \
                - 2.0 * average_hajek_sum_up / (average_hajek_sum_down**3) * average_hajek_cov)
        return se

    def _split_exposure_hajek_eht(self, node_id, df, probabilities, feature_set, max_attempt, eps, delta, outcome, rules, N, current_mse, c, covariates=None, naive=False, min_leaf_size=0):
        """
        the actual splitting implementation for non-separate tree; by recursion
        """
        b_feature = ''
        b_threshold = 0
        b_left = None
        b_right = None
        b_average_left_hajek = 0
        b_average_right_hajek = 0
        b_mse = 10000000000.0

        ranges = {}

        for feature in feature_set:
            import gc
            gc.collect()
            
            # find the more compact valid region
            upper = 1.
            lower = 0.
            for rule in rules:
                if rule[0] == feature:
                    if rule[1] == 0:
                        lower = np.maximum(rule[2], lower)
                    else:
                        upper = np.minimum(rule[2], upper)
                        
            for k in range(max_attempt):
                threshold = np.random.uniform(lower, upper)
                
                cz_l_1 = self._contain_zero(probabilities, rules+[(feature, 0, threshold)], eps, delta, treated=1)
                cz_r_1 = self._contain_zero(probabilities, rules+[(feature, 1, threshold)], eps, delta, treated=1)
                cz_l_0 = self._contain_zero(probabilities, rules+[(feature, 0, threshold)], eps, delta, treated=0)
                cz_r_0 = self._contain_zero(probabilities, rules+[(feature, 1, threshold)], eps, delta, treated=0)
                
                
                if np.mean(cz_l_1) > delta or np.mean(cz_r_1) > delta or np.mean(cz_r_0) > delta or np.mean(cz_r_0) > delta:
                    continue

                idxs_left = np.product([df[key] <= th for key, sign, th in rules if sign == 0] + \
                       [df[key] > th for key, sign, th in rules if sign == 1] + \
                        [df[feature] <= threshold],
                       axis=0) > 0

                idxs_right = np.product([df[key] <= th for key, sign, th in rules if sign == 0] + \
                       [df[key] > th for key, sign, th in rules if sign == 1] + \
                        [df[feature] > threshold],
                       axis=0) > 0

                left = df[idxs_left]
                right = df[idxs_right]
                
                # propensity score for left partition + ego treated
                propensities_left_1 = np.mean(np.product([probabilities[key][idxs_left] <= th for key, sign, th in rules if sign == 0] + \
                            [probabilities[key][idxs_left] > th for key, sign, th in rules if sign == 1] + \
                            [probabilities[feature][idxs_left] <= threshold] + \
                            [probabilities[self.treatment][idxs_left] == 1],
                           axis=0), axis=1)
                
                # propensity score for left partition + ego non treated
                propensities_left_0 = np.mean(np.product([probabilities[key][idxs_left] <= th for key, sign, th in rules if sign == 0] + \
                            [probabilities[key][idxs_left] > th for key, sign, th in rules if sign == 1] + \
                            [probabilities[feature][idxs_left] <= threshold] + \
                            [probabilities[self.treatment][idxs_left] == 0],
                           axis=0), axis=1)
                
                propensities_right_1 = np.mean(np.product([probabilities[key][idxs_right] <= th for key, sign, th in rules if sign == 0] + \
                            [probabilities[key][idxs_right] > th for key, sign, th in rules if sign == 1] + \
                            [probabilities[feature][idxs_right] > threshold] + \
                            [probabilities[self.treatment][idxs_right] == 1],
                           axis=0), axis=1)

                propensities_right_0 = np.mean(np.product([probabilities[key][idxs_right] <= th for key, sign, th in rules if sign == 0] + \
                            [probabilities[key][idxs_right] > th for key, sign, th in rules if sign == 1] + \
                            [probabilities[feature][idxs_right] > threshold] + \
                            [probabilities[self.treatment][idxs_right] == 0],
                           axis=0), axis=1)
                
                # filter those whose propensities scores are very small (This may lead to lose observations)
                idxs_left_filter = np.logical_and(propensities_left_1 > eps, propensities_left_0 > eps)
                left = left[idxs_left_filter]
                propensities_left_1 = propensities_left_1[idxs_left_filter]
                propensities_left_0 = propensities_left_0[idxs_left_filter]
                
                # filter those whose propensities scores are very small (This may lead to lose observations)
                idxs_right_filter = np.logical_and(propensities_right_1 > eps, propensities_right_0 > eps)
                right = right[idxs_right_filter]
                propensities_right_1 = propensities_right_1[idxs_right_filter]                
                propensities_right_0 = propensities_right_0[idxs_right_filter]                
                
                if np.mean(left[self.treatment]) == 0 or np.mean(left[self.treatment]) == 1 or \
                    np.mean(right[self.treatment]) == 0 or np.mean(right[self.treatment]) == 1:
                    continue
                
                if len(left) == 0 or len(right) == 0:
                    continue
                
                # The covariate implementation does not work as expected; should always be None
                # naive is True = conventional tree regression
                # naive is False = weighted least square for Hajek and weighted sum of square error
                if covariates is None:
                    if naive:
                        mod_left = sm.OLS(left[outcome], sm.add_constant(left[self.treatment]))
                    else:
                        mod_left = sm.WLS(left[outcome], sm.add_constant(left[[self.treatment]]), 
                        weights=1.0 / propensities_left_1 * left[self.treatment] + 1.0 / propensities_left_0 * (1-left[self.treatment]))
                else:
                    if naive:
                        mod_left = sm.OLS(left[outcome], sm.add_constant(left[self.treatment]+covariates))
                    else:
                        mod_left = sm.WLS(left[outcome], sm.add_constant(left[[self.treatment]+covariates]),
                    weights=1.0 / propensities_left_1 * left[self.treatment] + 1.0 / propensities_left_0 * (1-left[self.treatment]))
                res_left = mod_left.fit()

                if covariates is None:
                    if naive:
                        mod_right = sm.OLS(right[outcome], sm.add_constant(right[self.treatment]))
                    else:
                        mod_right = sm.WLS(right[outcome], sm.add_constant(right[self.treatment]), 
            weights=1.0 / propensities_right_1 * right[self.treatment] + 1.0 / propensities_right_0 * (1-right[self.treatment]))
                else:
                    if naive:
                        mod_right = sm.OLS(right[outcome], sm.add_constant(right[[self.treatment]+covariates]))
                    else:
                        mod_right = sm.WLS(right[outcome], sm.add_constant(right[[self.treatment]+covariates]), 
                    weights=1.0 / propensities_right_1 * right[self.treatment] + 1.0 / propensities_right_0 * (1-right[self.treatment]))
                        
                res_right = mod_right.fit()
                
                
                average_left_hajek = res_left.params[1] 
                average_right_hajek = res_right.params[1]

                average_left_hajek_se = res_left.bse[1]  
                average_right_hajek_se = res_right.bse[1]
                
                if naive:
                    mse_left = np.sum((res_left.resid) ** 2)
                    mse_right = np.sum((res_right.resid) ** 2)
                    mse = mse_left + mse_right                     
                else:
                    mse_left = np.sum((1.0 / propensities_left_1 * left[self.treatment] + 1.0 / propensities_left_0 * (1-left[self.treatment])) * 
                                      ((res_left.resid) ** 2))
                    mse_right = np.sum((1.0 / propensities_right_1 * right[self.treatment] + 1.0 / propensities_right_0 * (1-right[self.treatment])) * 
                                       ((res_right.resid) ** 2))
                    mse = mse_left * 1.0 * len(left)/(len(left)+len(right)) + mse_right * 1.0 * len(right)/(len(left)+len(right))

                if mse < current_mse - c and mse < b_mse:
                    if len(left) > min_leaf_size and len(right) > min_leaf_size:
                        # make sure leaf size is greater than the set value; default is > 0 which does not do anything
                        b_feature = feature
                        b_mse = mse
                        b_mse_left = mse_left
                        b_mse_right = mse_right
                        b_threshold = threshold
                        b_average_left_hajek = average_left_hajek
                        b_average_right_hajek = average_right_hajek
                        b_average_left_hajek_se = average_left_hajek_se
                        b_average_right_hajek_se = average_right_hajek_se
                        b_left = left
                        b_right = right
                        b_left_rules = rules + [(feature, 0, threshold)]
                        b_right_rules = rules + [(feature, 1, threshold)]
        
        result = {}
        if b_feature != '':
            # if find a valid partition
            result_left = self._split_exposure_hajek_eht(node_id*2, df, probabilities, feature_set, max_attempt, eps, delta, outcome, b_left_rules, N, b_mse_left, c)       
            result_right = self._split_exposure_hajek_eht(node_id*2+1, df, probabilities, feature_set, max_attempt, eps, delta, outcome, b_right_rules, N, b_mse_right, c)
            result['mse'] = result_left['mse'] * 1.0 * len(b_left)/(len(b_left)+len(b_right)) + \
                        result_right['mse'] * 1.0 * len(b_right)/(len(b_left)+len(b_right))
            result['feature'] = b_feature
            result['threshold'] = b_threshold
            result_left['hajek'] = b_average_left_hajek
            result_right['hajek'] = b_average_right_hajek
            result_left['hajek_se'] = b_average_left_hajek_se
            result_right['hajek_se'] = b_average_right_hajek_se
            result_left['N'] = len(b_left)
            result_right['N'] = len(b_right)
            result['left_result'] = result_left
            result['right_result'] = result_right
            return result
        else:
            result['mse'] = current_mse
            return result

    def _split_exposure_hajek(self, node_id, df, probabilities, feature_set, max_attempt, eps, delta, outcome, rules, N, current_mse, c, covariates=None, naive=False, treated=None, min_leaf_size=0):
        """
        the actual splitting implementation for separate tree; by recursion
        """
        b_feature = ''
        b_threshold = 0
        b_left = None
        b_right = None
        b_average_left_hajek = 0
        b_average_right_hajek = 0
        b_mse = 10000000000.0
        
        ranges = {}

        for feature in feature_set:
            import gc
            gc.collect()
            
            # find a more compact region
            upper = 1.
            lower = 0.
            for rule in rules:
                if rule[0] == feature:
                    if rule[1] == 0:
                        lower = np.maximum(rule[2], lower)
                    else:
                        upper = np.minimum(rule[2], upper)
                        
            for k in range(max_attempt):
                threshold = np.random.uniform(lower, upper)
                
                # make sure it is a valid split --- each observation should have non-trial (>eps) probability to belong to each partition
                cz_l = self._contain_zero(probabilities, rules+[(feature, 0, threshold)], eps, delta, treated)
                cz_r = self._contain_zero(probabilities, rules+[(feature, 1, threshold)], eps, delta, treated)
                
                if np.mean(cz_l) > delta or np.mean(cz_r) > delta:
                    continue
            
                idxs_left = np.product([df[key] <= th for key, sign, th in rules if sign == 0] + \
                       [df[key] > th for key, sign, th in rules if sign == 1] + \
                        [df[feature] <= threshold],
                       axis=0) > 0

                idxs_right = np.product([df[key] <= th for key, sign, th in rules if sign == 0] + \
                       [df[key] > th for key, sign, th in rules if sign == 1] + \
                        [df[feature] > threshold],
                       axis=0) > 0

                left = df[idxs_left]
                right = df[idxs_right]
                
                propensities_left = np.mean(np.product([probabilities[key][idxs_left] <= th for key, sign, th in rules if sign == 0] + \
                           [probabilities[key][idxs_left] > th for key, sign, th in rules if sign == 1] + \
                            [probabilities[feature][idxs_left] <= threshold] + [probabilities[self.treatment][idxs_left] == treated],
                           axis=0), axis=1)

                propensities_right = np.mean(np.product([probabilities[key][idxs_right] <= th for key, sign, th in rules if sign == 0] + \
                           [probabilities[key][idxs_right] > th for key, sign, th in rules if sign == 1] + \
                            [probabilities[feature][idxs_right] > threshold] + [probabilities[self.treatment][idxs_right] == treated],
                           axis=0), axis=1)
                
                # again, filter small propensities data points (usually should not filter or filter very few)
                idxs_left_filter = propensities_left > eps
                left = left[idxs_left_filter]
                propensities_left = propensities_left[idxs_left_filter]
                
                # again, filter small propensities data points (usually should not filter or filter very few)
                idxs_right_filter = propensities_right > eps
                right = right[idxs_right_filter]
                propensities_right = propensities_right[idxs_right_filter]                
                
                if len(left) == 0 or len(right) == 0:
                    continue
                
                # The covariate implementation does not work as expected; should always be None
                # naive is True = conventional tree regression
                # naive is False = weighted least square for Hajek and weighted sum of square error

                if covariates is None:
                    if naive:
                        mod_left = sm.OLS(left[outcome], np.ones(len(left)))
                    else:
                        mod_left = sm.WLS(left[outcome], np.ones(len(left)), weights=1.0 / propensities_left)
                else:
                    if naive:
                        mod_left = sm.OLS(right[outcome], np.ones(len(left)))
                    else:
                        mod_left = sm.WLS(left[outcome], sm.add_constant(left[covariates]), weights=1.0 / propensities_left)
                res_left = mod_left.fit()

                if covariates is None:
                    if naive:
                        mod_right = sm.OLS(right[outcome], np.ones(len(right)))
                    else:
                        mod_right = sm.WLS(right[outcome], np.ones(len(right)), weights=1.0 / propensities_right)
                else:
                    if naive:
                        mod_right = sm.OLS(right[outcome], sm.add_constant(right[covariates]))
                    else:
                        mod_right = sm.WLS(right[outcome], sm.add_constant(right[covariates]), weights=1.0 / propensities_right)
                        
                res_right = mod_right.fit()

                average_left_hajek = res_left.params[0] 
                average_right_hajek = res_right.params[0]

                average_left_hajek_se = self._hajek_se(left, propensities_left, outcome)
                average_right_hajek_se = self._hajek_se(right, propensities_right, outcome)
                
                if naive:
                    mse_left = np.sum((res_left.resid) ** 2)
                    mse_right = np.sum((res_right.resid) ** 2)
                    mse = mse_left + mse_right                     
                else:
                    mse_left = np.sum((1.0 / propensities_left) * ((res_left.resid) ** 2))
                    mse_right = np.sum((1.0 / propensities_right) * ((res_right.resid) ** 2))
                    mse = mse_left * 1.0 * len(left)/(len(left)+len(right)) + mse_right * 1.0 * len(right)/(len(left)+len(right))

                if mse < current_mse - c and mse < b_mse:
                    if len(left) > min_leaf_size and len(right) > min_leaf_size:
                        b_feature = feature
                        b_mse = mse
                        b_mse_left = mse_left
                        b_mse_right = mse_right
                        b_threshold = threshold
                        b_average_left_hajek = average_left_hajek
                        b_average_right_hajek = average_right_hajek
                        b_average_left_hajek_se = average_left_hajek_se
                        b_average_right_hajek_se = average_right_hajek_se
                        b_left_den = np.sum(1.0 / propensities_left)
                        b_right_den = np.sum(1.0 / propensities_right)
                        b_left = left
                        b_right = right
                        b_left_rules = rules + [(feature, 0, threshold)]
                        b_right_rules = rules + [(feature, 1, threshold)]
        
        result = {}
        if b_feature != '':
            # if find a valid partition
            result_left = self._split_exposure_hajek(node_id*2, df, probabilities, feature_set, max_attempt, eps, delta, outcome, b_left_rules, N, b_mse_left, c, treated=treated, min_leaf_size=min_leaf_size)   
            result_right = self._split_exposure_hajek(node_id*2+1, df, probabilities, feature_set, max_attempt, eps, delta, outcome, b_right_rules, N, b_mse_right, c, treated=treated, min_leaf_size=min_leaf_size)
            result['mse'] = result_left['mse'] * 1.0 * len(b_left)/(len(b_left)+len(b_right)) + \
                        result_right['mse'] * 1.0 * len(b_right)/(len(b_left)+len(b_right))
            result['feature'] = b_feature
            result['threshold'] = b_threshold
            result_left['hajek'] = b_average_left_hajek
            result_right['hajek'] = b_average_right_hajek
            result_left['hajek_se'] = b_average_left_hajek_se
            result_right['hajek_se'] = b_average_right_hajek_se
            result_left['N'] = len(b_left)
            result_right['N'] = len(b_right)
            result_left['den'] = b_left_den
            result_right['den'] = b_right_den
            result['left_result'] = result_left
            result['right_result'] = result_right
            return result
        else:
            result['mse'] = current_mse
            return result

    def _split_exposure_validate_hte(self, node_id, df_est, result, probabilities_est, rules, outcome, eps=0.005, covariates=None, naive=False):
        """
        estimation set for non-separate case
        """
        est_result = {}
        if 'left_result' in result:
            est_result['feature'] = result['feature']
            est_result['threshold'] = result['threshold']
            est_result['left_result'] = self._split_exposure_validate_hte(node_id*2, df_est, result['left_result'], probabilities_est, 
                                                       rules+[(result['feature'], 0, result['threshold'])], outcome, eps)
            est_result['right_result'] = self._split_exposure_validate_hte(node_id*2+1, df_est, result['right_result'], probabilities_est, 
                                                         rules+[(result['feature'], 1, result['threshold'])], outcome, eps)
        
        if rules:
            # if this is not the root
            idxs = np.product([df_est[key] <= th for key, sign, th in rules if sign == 0] + \
                   [df_est[key] > th for key, sign, th in rules if sign == 1],
                   axis=0) > 0
            dff = df_est[idxs]
            
            propensities_1 = np.mean(np.product([probabilities_est[key][idxs] <= th for key, sign, th in rules if sign == 0] + \
                   [probabilities_est[key][idxs] > th for key, sign, th in rules if sign == 1]+\
                    [probabilities_est[self.treatment][idxs] == 1],
                   axis=0), axis=1)
            
            propensities_0 = np.mean(np.product([probabilities_est[key][idxs] <= th for key, sign, th in rules if sign == 0] + \
                   [probabilities_est[key][idxs] > th for key, sign, th in rules if sign == 1]+\
                   [probabilities_est[self.treatment][idxs] == 0],
                   axis=0), axis=1)
            
        else:
            # if this is the root
            idxs = np.ones(len(df_est)).astype(bool)
            dff = df_est[idxs]
            propensities_1 = np.ones(len(dff)) * np.mean(dff[self.treatment])
            propensities_0 = np.ones(len(dff)) * (1 - np.mean(dff[self.treatment]))
        
        idxs_filter = np.logical_and(propensities_1 > eps, propensities_0 > eps)
        dff = dff[idxs_filter]
        propensities_1 = propensities_1[idxs_filter]
        propensities_0 = propensities_0[idxs_filter]
        
        if covariates is None:
            if naive:
                mod = sm.OLS(dff[outcome], sm.add_constant(dff[self.treatment]))
            else:
                mod = sm.WLS(dff[outcome], sm.add_constant(dff[self.treatment]), 
                             weights=1.0 / propensities_1 * dff[self.treatment] + 1.0 / propensities_0 * (1-dff[self.treatment]))
        else:
            if naive:
                mod = sm.OLS(dff[outcome], sm.add_constant(dff[[self.treatment]+[covariates]]))
            else:
                mod = sm.WLS(dff[outcome], sm.add_constant(dff[[self.treatment]+[covariates]]), 
                weights=1.0 / propensities_1 * dff[self.treatment] + 1.0 / propensities_0 * (1-dff[self.treatment]))
                
        res = mod.fit()

        if naive:
            mse = np.sum((res.resid ** 2))
        else:
            mse = np.sum((res.resid ** 2) * (1.0 / propensities_1 * dff[self.treatment] + 1.0 / propensities_0 * (1-dff[self.treatment]))) 
        
        average_hajek = res.params[1]
        average_hajek_se = res.bse[1] # dff[outcome].std() / np.sqrt(len(dff)-1)
        average_hajek_se = res.bse[1]
        
        est_result['hajek'] = average_hajek
        est_result['hajek_se'] = average_hajek_se
        est_result['mse'] = mse
        est_result['N'] = len(dff)
        return est_result
        
    def _split_exposure_validate(self, node_id, df_est, result, 
                                 probabilities_est, rules, outcome, eps=0.005, covariates=None, 
                                 naive=False, treated=None):

        est_result = {}
        if 'left_result' in result:
            est_result['feature'] = result['feature']
            est_result['threshold'] = result['threshold']
            est_result['left_result'] = self._split_exposure_validate(node_id*2, df_est, result['left_result'], probabilities_est, 
                                                       rules+[(result['feature'], 0, result['threshold'])], outcome, eps, treated=treated)
            est_result['right_result'] = self._split_exposure_validate(node_id*2+1, df_est, result['right_result'], probabilities_est, 
                                                         rules+[(result['feature'], 1, result['threshold'])], outcome, eps, treated=treated)
        
        if rules:
            idxs = np.product([df_est[key] <= th for key, sign, th in rules if sign == 0] + \
                   [df_est[key] > th for key, sign, th in rules if sign == 1], axis=0) > 0
            dff = df_est[idxs]
            propensities = np.mean(np.product([probabilities_est[key][idxs] <= th for key, sign, th in rules if sign == 0] + \
                   [probabilities_est[key][idxs] > th for key, sign, th in rules if sign == 1] + \
                   [probabilities_est[self.treatment][idxs] == treated],
                   axis=0), axis=1)

        else:
            idxs = np.ones(len(df_est)).astype(bool)
            dff = df_est[idxs]
            propensities = np.ones(len(dff)) * 0.5
        
        idxs_filter = propensities > eps
        dff = dff[idxs_filter]
        propensities = propensities[idxs_filter]
        
        if covariates is None:
            if naive:
                mod = sm.OLS(dff[outcome], np.ones(len(dff)))
            else:
                mod = sm.WLS(dff[outcome], np.ones(len(dff)), weights=1.0 / propensities)
        else:
            if naive:
                mod = sm.OLS(dff[outcome], sm.add_constant(dff[covariates]))
            else:
                mod = sm.WLS(dff[outcome], sm.add_constant(dff[covariates]), weights=1.0 / propensities)
                
        res = mod.fit()
        if naive:
            mse = np.sum((res.resid ** 2))
        else:
            mse = np.sum((res.resid ** 2) * 1.0 / propensities)
        
        
        average_hajek = res.params[0]
        if node_id == 1:
            average_hajek_se = dff[outcome].std() / np.sqrt(len(dff)-1)
        else:
            average_hajek_se = self._hajek_se(dff, propensities, outcome)
        
        est_result['hajek'] = average_hajek
        est_result['hajek_se'] = average_hajek_se
        est_result['mse'] = mse
        est_result['N'] = len(dff)
        return est_result
        
    def estimate_exposure_hajek(self, train_result_separate, indirect_space, outcome, eps=0.005, cv=False, df_est=None, probabilities=None, covariates=None, separate=True):
        """
        train_result_separate: result from training
        indirect_space: feature space (consistent with training input)
        outcome: (consistent with training input)
        eps:  (consistent with training input)
        cv=False: leave it 
        df_est=None: leave it 
        probabilities=None: leave it 
        covariates=None: leave it 
        separate=True: separate trees.
        """
        if separate:
            if cv == False:
                # if find a valid partition for T == 1 or 0 separately
                df_est = self.df_est
                probabilities = self.probabilities

                idx_est_1 = np.array(self.df_est[self.treatment] == 1)
                idx_est_0 = np.array(self.df_est[self.treatment] == 0)

                df_est_1 = self.df_est[idx_est_1]
                df_est_0 = self.df_est[idx_est_0]

                probabilities_est_1 = {}
                for key in indirect_space+[self.treatment]:
                    probabilities_est_1[key] = self.probabilities[key][self.idx_est]
                    probabilities_est_1[key] = probabilities_est_1[key][idx_est_1.astype(bool)]

                probabilities_est_0 = {}
                for key in indirect_space+[self.treatment]:
                    probabilities_est_0[key] = self.probabilities[key][self.idx_est]
                    probabilities_est_0[key] = probabilities_est_0[key][idx_est_0.astype(bool)]
            else:
                idx_est_1 = np.array(df_est[self.treatment] == 1)
                idx_est_0 = np.array(df_est[self.treatment] == 0)

                df_est_1 = df_est[idx_est_1]
                df_est_0 = df_est[idx_est_0]

                probabilities_est_1 = {}
                for key in indirect_space:
                    probabilities_est_1[key] = probabilities[key][idx_est_1]

                probabilities_train_0 = {}
                for key in indirect_space:
                    probabilities_est_0[key] = probabilities[key][idx_est_0]

            est_result_separate = {} 
            est_result_separate[1] = self._split_exposure_validate(1, df_est_1, train_result_separate[1], probabilities_est_1, [], outcome, eps, covariates, treated=1)
            est_result_separate[0] = self._split_exposure_validate(1, df_est_0, train_result_separate[0], probabilities_est_0, [], outcome, eps, covariates, treated=0)
            self.est_result_separate = est_result_separate
            return est_result_separate
        else:
            if cv == False:
                # if find a valid partition for T == 1 or 0 separately
                df_est = self.df_est
                probabilities_est = {}
                for key in indirect_space+[self.treatment]:
                        probabilities_est[key] = self.probabilities[key][self.idx_est.astype(bool)]
            else:
                pass
                #TODO
            est_result_separate_eht = {} 
            est_result_separate_eht = self._split_exposure_validate_hte(1, df_est, train_result_separate, probabilities_est, [], outcome, eps, covariates)
            self.est_result_separate_eht = est_result_separate_eht
            return est_result_separate_eht
        
    def split_exposure_hajek(self, separate, outcome, indirect_space, min_variance_reduct=0.0, max_attempt=20, eps=0.0, delta=0.0, cv=False, df_train=None, probabilities=None, covariates=None, naive=False, min_leaf_size=0):
        """
        The API for spitting
        separate: True=separate trees
        outcome: outcome variable
        indirect_space: a list of features used to partition
        min_variance_reduct: minimum variance reduction in each partition, only partition if reduction is significantly large
        max_attempt: sample threshold -- a larger value tend to over fit more
        eps: avoid non-zero or zero-trivial probability
        delta: avoid non-zero or zero-trivial probability
        cv: cross validate for min_variance
        df_train.. leave it as None
        probabilities.. leave it as None
        covariates: leave it 
        naive: conventional tree algo
        min_leaf_size= default is 0; we can adjust it (e.g. 1000)
        """
        if separate == True:
            if cv == False:
                # if find a valid partition for T == 1 or 0 separately
                df_train = self.df_train
                probabilities = self.probabilities

                idx_tr_1 = np.array(df_train[self.treatment] == 1)
                idx_tr_0 = np.array(df_train[self.treatment] == 0)

                df_train_1 = df_train[idx_tr_1]
                df_train_0 = df_train[idx_tr_0]

                probabilities_train_1 = {}
                for key in indirect_space+[self.treatment]:
                    probabilities_train_1[key] = probabilities[key][self.idx_tr]
                    probabilities_train_1[key] = probabilities_train_1[key][idx_tr_1.astype(bool)]
                
                probabilities_train_0 = {}
                for key in indirect_space+[self.treatment]:
                    probabilities_train_0[key] = self.probabilities[key][self.idx_tr]
                    probabilities_train_0[key] = probabilities_train_0[key][idx_tr_0.astype(bool)]
            else:
                idx_tr_1 = np.array(df_train[self.treatment] == 1)
                idx_tr_0 = np.array(df_train[self.treatment] == 0)

                df_train_1 = df_train[idx_tr_1]
                df_train_0 = df_train[idx_tr_0]

                probabilities_train_1 = {}
                for key in indirect_space+[self.treatment]:
                        probabilities_train_1[key] = probabilities[key][idx_tr_1]
                
                probabilities_train_0 = {}
                for key in indirect_space+[self.treatment]:
                        probabilities_train_0[key] = probabilities[key][idx_tr_0]
            
            if covariates is None:
                mod_1 = sm.WLS(df_train_1[outcome], np.ones(len(df_train_1)))
            else:
                mod_1 = sm.WLS(df_train_1[outcome], sm.add_constant(df_train_1))
            res_1 = mod_1.fit()
            total_mse_1 = np.sum(res_1.resid ** 2) * 2
            
            train_result_separate = {}
            train_result_separate[1] = self._split_exposure_hajek(1, df_train_1, probabilities_train_1, indirect_space, max_attempt, eps, delta, outcome, [], np.sum(df_train_1), total_mse_1, min_variance_reduct, covariates, naive, treated=1, min_leaf_size=min_leaf_size)
            
            train_result_separate[1]['N'] = len(df_train_1)
            train_result_separate[1]['hajek'] = df_train_1[outcome].mean()
            train_result_separate[1]['hajek_se'] = df_train_1[outcome].std() / np.sqrt(len(df_train_1[outcome])-1)

            if covariates is None:
                mod_0 = sm.WLS(df_train_0[outcome], np.ones(len(df_train_0)))
            else:
                mod_0 = sm.WLS(df_train_0[outcome], sm.add_constant(df_train_0))
                
            res_0 = mod_0.fit()
            total_mse_0 = np.sum(res_0.resid ** 2) * 2  
            train_result_separate[0] = self._split_exposure_hajek(1, df_train_0, probabilities_train_0, indirect_space, max_attempt, eps, delta, outcome, [], np.sum(df_train_0), total_mse_0, min_variance_reduct, covariates, naive, treated=0, min_leaf_size=min_leaf_size)
            train_result_separate[0]['N'] = len(df_train_0)
            train_result_separate[0]['hajek'] = df_train_0[outcome].mean()
            train_result_separate[0]['hajek_se'] = df_train_0[outcome].std() / np.sqrt(len(df_train_0[outcome])-1)
            self.train_result_separate = train_result_separate
            return train_result_separate
        else:
            # TODO: Finish covariates estimation
            if covariates is None:
                
                df_train = self.df_train                
                probabilities = self.probabilities
                probabilities_train = {}
                for key in indirect_space+[self.treatment]:
                    probabilities_train[key] = probabilities[key][self.idx_tr]
                mod = sm.WLS(df_train[outcome], sm.add_constant(df_train[self.treatment]))
            else:
                pass
                #TODO
                
            res = mod.fit()
            total_mse = np.sum(res.resid ** 2) * 2
            
            train_result_eht = {}
            train_result_eht = self._split_exposure_hajek_eht(1, df_train, probabilities_train, indirect_space, max_attempt, eps, delta, outcome, [], np.sum(df_train), total_mse, min_variance_reduct, covariates, naive, min_leaf_size=min_leaf_size)
            train_result_eht['N'] = len(df_train)
            train_result_eht['hajek'] = res.params[1]
            train_result_eht['hajek_se'] = res.bse[1] 
            return train_result_eht
        
    def _plot_tree(self, est_result_separate, node_id, prefix):
        if node_id > 1 and node_id % 2 == 0:
            sign = '<='
        elif node_id > 1 and node_id % 2 == 1:
            sign = '> '
        else:
            sign = ''
        if 'left_result' in est_result_separate:
            print('%s%s(%d) split %s at %f, n=%d, avg=%f, se=%f' % (prefix, sign, node_id, est_result_separate['feature'], 
                est_result_separate['threshold'], est_result_separate['N'], est_result_separate['hajek'], est_result_separate['hajek_se']))
            self._plot_tree(est_result_separate['left_result'], node_id*2, prefix+'\t')
            self._plot_tree(est_result_separate['right_result'], node_id*2+1, prefix+'\t')
        else:
            print('%s%s(%d) terminate, n=%d, avg=%f, se=%f' % (prefix, sign, node_id, 
                        est_result_separate['N'], est_result_separate['hajek'], est_result_separate['hajek_se']))
    
    def plot_tree(self, result=None):
        if 1 in result:
            print('treated')
            self._plot_tree(result[1], 1, '')
            print('non-treated')
            self._plot_tree(result[0], 1, '')
        else:
            self._plot_tree(result, 1, '')
            
    def split_exposure_hajek_cv(self, separate, outcome, indirect_space, cv_fold=5, max_attempt=20, eps=0.0, delta=0.0, min_variance_reduct=[0.0, 5.0, 10.0, 20.0, 50.0]):
        """
        currently only support cv for min_variance_reduct; need further debugging
        """
        
        randints = np.random.randint(0, cv_fold, len(self.df_train))
        b_mean = 10000000000
        probabilities = self.probabilities
        for c in min_variance_reduct:
            mse_map = {}
            randints = np.random.randint(0, 5, len(self.df_train))
            res = []
            for cv in range(5):
                idx_tr_tr = randints != cv
                df_tr_tr = self.df_train[idx_tr_tr]
                df_tr_cv = self.df_train[np.logical_not(idx_tr_tr)]
                
                total_mse = np.sum((self.df_train[idx_tr_tr][outcome] - np.mean(self.df_train[idx_tr_tr][outcome])) ** 2)
                
                probabilities_tr_tr = {}
                for key in indirect_space:
                    probabilities_tr_tr[key] = probabilities[key][self.idx_tr][idx_tr_tr.astype(bool)]

                probabilities_tr_cv = {}
                for key in indirect_space:
                    probabilities_tr_cv[key] = probabilities[key][self.idx_tr][np.logical_not(idx_tr_tr)]

                result = self.split_exposure_hajek(separate, outcome, indirect_space, min_variance_reduct=c, max_attempt=max_attempt, eps=eps, delta=delta, cv=True, df_train=df_tr_tr, probabilities=probabilities_tr_tr)
                
                # res.append(self.split_exposure_hajek(separate, outcome, indirect_space, min_variance_reduct=c, max_attempt=max_attempt, eps=eps, delta=delta, cv=True, df_train=df_tr_cv, probabilities=probabilities_tr_cv, naive=False))
                
                self.estimate_exposure_hajek(result, indirect_space, outcome, eps=0.005, cv=False, df_est=None, probabilities=None, covariates=None, separate=True)
            print(c, np.mean([r[1]['mse'] + r[0]['mse'] for r in res]))
            if np.mean([r[1]['mse'] + r[0]['mse'] for r in res]) < b_mean:
                b_c = c
                b_mean = np.mean([r[1]['mse'] + r[0]['mse'] for r in res])
                
        return b_c, b_mean

In [323]:
# pure cutoff
data['y1'] = data['neighbor'] * 0.1 + data['gender'] * 1 + \
            data['assignment'] * (data['bbb_2_normalized'] > 0.7).astype(float) * 2  +\
            np.random.normal(0, 1, len(data))

# structural diversity is causal
data['y2'] = \
    data['neighbor'] * 0.1 + data['gender'] * 1 + \
    data['structural_diversity'] + \
    data['assignment'] * data['structural_diversity'] * 1 + \
    np.random.normal(0, 1, len(data))

# structural diversity is correlational
data['y3'] = \
    data['neighbor'] * 0.1 + data['gender'] * 1 + \
    data['structural_diversity_1'] + \
    data['assignment'] * data['structural_diversity_1'] * 1 + \
    np.random.normal(0, 1, len(data))


# make naive tree fail
data['y4'] = \
    5 * np.mean(new_probabilities['bbb_2_normalized'] <= 0.3, axis=1) +\
    np.random.normal(0, 1, len(data)) 

# irrelevant covariates
data['y5'] = data['neighbor'] + np.random.normal(0, 1, len(data)) 

/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in less_equal


In [324]:
idx = np.logical_and(np.logical_and(data['bbb_0'] + data['bbb_1'] + data['bbb_2'] > 0,
                     data['bbn_0'] + data['bbn_1'] + data['bbn_2'] > 0),
                     data['open_square_0']+data['open_square_1']+data['open_square_2']+data['open_square_3'] > 0)

In [325]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}

input_features = [       
           'bb_1_normalized', 'bb_0_normalized'
            #     ,
            #            'bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
            #            'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized', 
            #            'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
        ]
    
for key in ['assignment']+input_features:
        probabilities_[key] = new_probabilities[key][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3', 'y4', 'y5']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(True, outcome, input_features, 
                                                           min_variance_reduct=0, max_attempt=30, eps=0.0, 
                                                           delta=0.0, min_leaf_size=1000)

    partition.plot_tree(train_result_separate)
    
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    input_features, outcome, eps=0.0)
    
    partition.plot_tree(est_result_separate)

#
# y1
# treated
# (1) split bb_1_normalized at 0.596260, n=24249, avg=2.920641, se=0.009350
# 	<=(2) terminate, n=6915, avg=2.023325, se=0.019241
# 	> (3) terminate, n=17333, avg=3.213367, se=0.008418
# non-treated
# (1) terminate, n=24509, avg=1.500002, se=0.007297
# treated
# (1) split bb_1_normalized at 0.596260, n=24043, avg=2.930951, se=0.009366
# 	<=(2) terminate, n=6859, avg=2.047228, se=0.018855
# 	> (3) terminate, n=17184, avg=3.213723, se=0.008565
# non-treated
# (1) terminate, n=24290, avg=1.497633, se=0.007250
# y2
# treated
# (1) split bb_0_normalized at 0.416431, n=24249, avg=10.179848, se=0.023873
# 	<=(2) terminate, n=17343, avg=11.131863, se=0.025042
# 	> (3) terminate, n=6905, avg=8.100665, se=0.038821
# non-treated
# (1) split bb_1_normalized at 0.339317, n=24509, avg=4.530948, se=0.013113
# 	<=(2) terminate, n=14368, avg=3.638521, se=0.012202
# 	> (3) terminate, n=10141, avg=5.682745, se=0.019367
# treated
# (1) split bb_0_normalized at 0.416431, n=24043, avg=10.162427, se=0.023864
# 	<=(2) terminate, n=17188, avg=11.141049, se=0.024977
# 	> (3) terminate, n=6855, avg=8.046053, se=0.040634
# non-treated
# (1) split bb_1_normalized at 0.339317, n=24290, avg=4.535775, se=0.012973
# 	<=(2) terminate, n=14172, avg=3.661448, se=0.012278
# 	> (3) terminate, n=10118, avg=5.639381, se=0.020136
# y3
# treated
# (1) split bb_1_normalized at 0.848538, n=24249, avg=15.500158, se=0.029587
# 	<=(2) terminate, n=20351, avg=15.493240, se=0.027753
# 	> (3) terminate, n=3878, avg=15.330909, se=0.110452
# non-treated
# (1) terminate, n=24509, avg=8.466258, se=0.016146
# treated
# (1) split bb_1_normalized at 0.848538, n=24043, avg=15.476402, se=0.029570
# 	<=(2) terminate, n=20034, avg=15.458408, se=0.027996
# 	> (3) terminate, n=3964, avg=15.322069, se=0.106752
# non-treated
# (1) terminate, n=24290, avg=8.508087, se=0.016216
# y4
# treated
# (1) terminate, n=24249, avg=2.528752, se=0.006784
# non-treated
# (1) terminate, n=24509, avg=2.522605, se=0.006760
# treated
# (1) terminate, n=24043, avg=2.519914, se=0.006775
# non-treated
# (1) terminate, n=24290, avg=2.523007, se=0.006781
# y5
# /Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in sqrt
# treated
# (1) terminate, n=24249, avg=10.056291, se=0.014096
# non-treated
# (1) terminate, n=24509, avg=10.046116, se=0.013738
# treated
# (1) terminate, n=24043, avg=10.064965, se=0.013910
# non-treated
# (1) terminate, n=24290, avg=10.066351, se=0.013860


y1
treated
(1) split bb_1_normalized at 0.596260, n=24249, avg=2.920641, se=0.009350
	<=(2) terminate, n=6915, avg=2.023325, se=0.019241
	> (3) terminate, n=17333, avg=3.213367, se=0.008418
non-treated
(1) terminate, n=24509, avg=1.500002, se=0.007297
treated
(1) split bb_1_normalized at 0.596260, n=24043, avg=2.930951, se=0.009366
	<=(2) terminate, n=6859, avg=2.047228, se=0.018855
	> (3) terminate, n=17184, avg=3.213723, se=0.008565
non-treated
(1) terminate, n=24290, avg=1.497633, se=0.007250
y2
treated
(1) split bb_0_normalized at 0.416431, n=24249, avg=10.179848, se=0.023873
	<=(2) terminate, n=17343, avg=11.131863, se=0.025042
	> (3) terminate, n=6905, avg=8.100665, se=0.038821
non-treated
(1) split bb_1_normalized at 0.339317, n=24509, avg=4.530948, se=0.013113
	<=(2) terminate, n=14368, avg=3.638521, se=0.012202
	> (3) terminate, n=10141, avg=5.682745, se=0.019367
treated
(1) split bb_0_normalized at 0.416431, n=24043, avg=10.162427, se=0.023864
	<=(2) terminate, n=17188, avg=1

/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in sqrt


treated
(1) terminate, n=24249, avg=10.056291, se=0.014096
non-treated
(1) terminate, n=24509, avg=10.046116, se=0.013738
treated
(1) terminate, n=24043, avg=10.064965, se=0.013910
non-treated
(1) terminate, n=24290, avg=10.066351, se=0.013860


In [319]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}

input_features = [       
           'bb_1_normalized', 'bb_0_normalized',
           'bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
           'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized', 
           'open_square_0_normalized', 'open_square_1_normalized', 
           'open_square_2_normalized', 'open_square_3_normalized'
        ]
    
for key in ['assignment']+input_features:
        probabilities_[key] = new_probabilities[key][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3', 'y4', 'y5']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(True, outcome, input_features, 
                                                           min_variance_reduct=20.0, max_attempt=30, eps=0.0, delta=0.0, 
                                                           min_leaf_size=1000)

    partition.plot_tree(train_result_separate)
    
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    input_features, outcome, eps=0.0)
    
    partition.plot_tree(est_result_separate)
# y1
# treated
# (1) split bbb_2_normalized at 0.694570, n=24000, avg=2.924203, se=0.009275
# 	<=(2) split bbb_1_normalized at 0.402063, n=7002, avg=1.536370, se=0.013849
# 		<=(4) split bbn_1_normalized at 0.530563, n=4188, avg=1.549382, se=0.027197
# 			<=(8) terminate, n=1014, avg=1.666844, se=0.043752
# 			> (9) terminate, n=3074, avg=1.512184, se=0.031131
# 		> (5) terminate, n=2752, avg=1.495867, se=0.029824
# 	> (3) terminate, n=16998, avg=3.495275, se=0.007222
# non-treated
# (1) terminate, n=24453, avg=1.508827, se=0.007256
# treated
# (1) split bbb_2_normalized at 0.694570, n=24292, avg=2.926250, se=0.009271
# 	<=(2) split bbb_1_normalized at 0.402063, n=7041, avg=1.517720, se=0.014293
# 		<=(4) split bbn_1_normalized at 0.530563, n=4171, avg=1.553935, se=0.027513
# 			<=(8) terminate, n=1027, avg=1.566069, se=0.041693
# 			> (9) terminate, n=3018, avg=1.542752, se=0.032128
# 		> (5) terminate, n=2810, avg=1.498929, se=0.028222
# 	> (3) terminate, n=17251, avg=3.503794, se=0.007185
# non-treated
# (1) terminate, n=24346, avg=1.505709, se=0.007334
# y2
# /Users/yuany/Library/Python/3.7/bin/ipython:55: RuntimeWarning: invalid value encountered in sqrt
# treated
# (1) split open_square_3_normalized at 0.065991, n=24000, avg=10.166644, se=0.023870
# 	<=(2) split bbn_1_normalized at 0.636179, n=6715, avg=7.437064, se=0.032801
# 		<=(4) terminate, n=4881, avg=7.467024, se=0.046208
# 		> (5) terminate, n=1790, avg=5.763393, se=0.071804
# 	> (3) terminate, n=17285, avg=11.167805, se=0.021671
# non-treated
# (1) split bb_0_normalized at 0.695436, n=24453, avg=4.537264, se=0.013011
# 	<=(2) terminate, n=12354, avg=5.518526, se=0.015470
# 	> (3) terminate, n=12099, avg=3.449118, se=0.013746
# treated
# (1) split open_square_3_normalized at 0.065991, n=24292, avg=10.166942, se=0.023758
# 	<=(2) split bbn_1_normalized at 0.636179, n=6740, avg=7.407410, se=0.032598
# 		<=(4) terminate, n=4872, avg=7.466314, se=0.046471
# 		> (5) terminate, n=1823, avg=5.858726, se=0.076375
# 	> (3) terminate, n=17552, avg=11.169906, se=0.021632
# non-treated
# (1) split bb_0_normalized at 0.695436, n=24346, avg=4.529192, se=0.013053
# 	<=(2) terminate, n=12264, avg=5.488249, se=0.016379
# 	> (3) terminate, n=12082, avg=3.447003, se=0.013860
# y3
# treated
# (1) split open_square_2_normalized at 0.556849, n=24000, avg=15.486924, se=0.029653
# 	<=(2) split open_square_0_normalized at 0.150654, n=18093, avg=15.452211, se=0.030602
# 		<=(4) terminate, n=16379, avg=15.442717, se=0.032768
# 		> (5) terminate, n=1634, avg=16.112239, se=0.135012
# 	> (3) split open_square_3_normalized at 0.127685, n=5875, avg=13.551283, se=0.062676
# 		<=(6) terminate, n=1223, avg=10.948792, se=0.103875
# 		> (7) terminate, n=4631, avg=13.615652, se=0.072070
# non-treated
# (1) terminate, n=24453, avg=8.485925, se=0.016212
# treated
# (1) split open_square_2_normalized at 0.556849, n=24292, avg=15.481324, se=0.029419
# 	<=(2) split open_square_0_normalized at 0.150654, n=18271, avg=15.435781, se=0.030447
# 		<=(4) terminate, n=16653, avg=15.451829, se=0.032620
# 		> (5) terminate, n=1549, avg=15.834276, se=0.133377
# 	> (3) split open_square_3_normalized at 0.127685, n=6001, avg=13.681437, se=0.066993
# 		<=(6) terminate, n=1216, avg=11.113086, se=0.107962
# 		> (7) terminate, n=4765, avg=13.686914, se=0.079912
# non-treated
# (1) terminate, n=24346, avg=8.498186, se=0.016178
# y4
# treated
# (1) split open_square_2_normalized at 0.577629, n=24000, avg=2.539997, se=0.006853
# 	<=(2) terminate, n=19831, avg=2.560876, se=0.006283
# 	> (3) split bb_0_normalized at 0.260502, n=4120, avg=2.539796, se=0.027127
# 		<=(6) terminate, n=2943, avg=2.572890, se=0.020015
# 		> (7) terminate, n=1140, avg=2.512248, se=0.045796
# non-treated
# (1) terminate, n=24453, avg=2.523293, se=0.006778
# treated
# (1) split open_square_2_normalized at 0.577629, n=24292, avg=2.522186, se=0.006784
# 	<=(2) terminate, n=20079, avg=2.539551, se=0.006210
# 	> (3) split bb_0_normalized at 0.260502, n=4164, avg=2.544055, se=0.027542
# 		<=(6) terminate, n=3011, avg=2.548764, se=0.023057
# 		> (7) terminate, n=1119, avg=2.530935, se=0.047084
# non-treated
# (1) terminate, n=24346, avg=2.532521, se=0.006821
# y5
# treated
# (1) split open_square_2_normalized at 0.557960, n=24000, avg=10.082184, se=0.014065
# 	<=(2) terminate, n=18296, avg=10.065881, se=0.013977
# 	> (3) split open_square_1_normalized at 0.152125, n=5671, avg=9.437733, se=0.033147
# 		<=(6) terminate, n=2408, avg=9.181124, se=0.056617
# 		> (7) terminate, n=3209, avg=9.493790, se=0.044223
# non-treated
# (1) terminate, n=24453, avg=10.060567, se=0.013837
# treated
# (1) split open_square_2_normalized at 0.557960, n=24292, avg=10.056364, se=0.013981
# 	<=(2) terminate, n=18463, avg=10.036745, se=0.014077
# 	> (3) split open_square_1_normalized at 0.152125, n=5805, avg=9.453527, se=0.033766
# 		<=(6) terminate, n=2427, avg=9.196167, se=0.056113
# 		> (7) terminate, n=3314, avg=9.475883, se=0.046019
# non-treated
# (1) terminate, n=24346, avg=10.052287, se=0.013737


y1


/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in sqrt


treated
(1) split bbb_2_normalized at 0.697774, n=24006, avg=4.899833, se=0.018773
	<=(2) split bbb_1_normalized at 0.495707, n=7058, avg=1.512714, se=0.014812
		<=(4) terminate, n=5006, avg=1.522125, se=0.023536
		> (5) terminate, n=1988, avg=1.482362, se=0.036691
	> (3) terminate, n=16948, avg=6.289569, se=0.013982
non-treated
(1) terminate, n=24476, avg=1.533619, se=0.007445
treated
(1) split bbb_2_normalized at 0.697774, n=24286, avg=4.951749, se=0.018585
	<=(2) split bbb_1_normalized at 0.495707, n=6985, avg=1.544738, se=0.014476
		<=(4) terminate, n=4954, avg=1.573797, se=0.024583
		> (5) terminate, n=1980, avg=1.437694, se=0.035797
	> (3) terminate, n=17301, avg=6.322474, se=0.013657
non-treated
(1) terminate, n=24323, avg=1.542266, se=0.007492
y2
treated
(1) split open_square_3_normalized at 0.064866, n=24006, avg=10.191062, se=0.023890
	<=(2) split open_square_1_normalized at 0.553695, n=6572, avg=7.448211, se=0.033406
		<=(4) terminate, n=4483, avg=7.876422, se=0.043557
		> (

In [320]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}

input_features = [       
           'bb_1_normalized', 'bb_0_normalized',
           'bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
           'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized', 
           'open_square_0_normalized', 'open_square_1_normalized', 
           'open_square_2_normalized', 'open_square_3_normalized'
        ]
    
for key in ['assignment']+input_features:
        probabilities_[key] = new_probabilities[key][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3', 'y4', 'y5']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(True, outcome, input_features, 
                                                           min_variance_reduct=0.0, max_attempt=10, eps=0.0, delta=0.0, 
                                                           min_leaf_size=1000)

    partition.plot_tree(train_result_separate)
    
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    input_features, outcome, eps=0.0)
    
    partition.plot_tree(est_result_separate)
    
    
# y1
# /Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in sqrt
# treated
# (1) split bbb_2_normalized at 0.690091, n=24087, avg=3.808138, se=0.014507
# 	<=(2) split bbb_1_normalized at 0.467876, n=6976, avg=1.505662, se=0.014784
# 		<=(4) split bbn_0_normalized at 0.315332, n=4889, avg=1.517870, se=0.024327
# 			<=(8) terminate, n=4002, avg=1.549121, se=0.027253
# 			> (9) terminate, n=741, avg=1.508550, se=0.047408
# 		> (5) split open_square_3_normalized at 0.201527, n=2017, avg=1.453078, se=0.037204
# 			<=(10) terminate, n=1511, avg=1.431420, se=0.042567
# 			> (11) terminate, n=401, avg=1.464826, se=0.062757
# 	> (3) terminate, n=17111, avg=4.729690, se=0.011975
# non-treated
# (1) split open_square_0_normalized at 0.215481, n=24513, avg=1.546998, se=0.007393
# 	<=(2) terminate, n=14735, avg=1.560975, se=0.008313
# 	> (3) terminate, n=9778, avg=1.526266, se=0.010743
# treated
# (1) split bbb_2_normalized at 0.690091, n=24205, avg=3.815246, se=0.014398
# 	<=(2) split bbb_1_normalized at 0.467876, n=6953, avg=1.502430, se=0.014388
# 		<=(4) split bbn_0_normalized at 0.315332, n=4924, avg=1.518344, se=0.024167
# 			<=(8) terminate, n=4108, avg=1.555798, se=0.028296
# 			> (9) terminate, n=686, avg=1.397646, se=0.050585
# 		> (5) split open_square_3_normalized at 0.201527, n=1984, avg=1.472725, se=0.033446
# 			<=(10) terminate, n=1481, avg=1.508631, se=0.038511
# 			> (11) terminate, n=401, avg=1.347969, se=0.063562
# 	> (3) terminate, n=17252, avg=4.736733, se=0.011850
# non-treated
# (1) split open_square_0_normalized at 0.215481, n=24286, avg=1.529098, se=0.007499
# 	<=(2) terminate, n=14648, avg=1.541490, se=0.008412
# 	> (3) terminate, n=9638, avg=1.505644, se=0.011098
# y2
# treated
# (1) terminate, n=24087, avg=10.159172, se=0.023892
# non-treated
# (1) split bbn_2_normalized at 0.077632, n=24513, avg=4.519019, se=0.012924
# 	<=(2) terminate, n=12083, avg=3.416096, se=0.013473
# 	> (3) terminate, n=12430, avg=5.507363, se=0.015120
# treated
# (1) terminate, n=24205, avg=10.172325, se=0.023789
# non-treated
# (1) split bbn_2_normalized at 0.077632, n=24286, avg=4.531831, se=0.013137
# 	<=(2) terminate, n=11843, avg=3.397231, se=0.013897
# 	> (3) terminate, n=12443, avg=5.531270, se=0.015213
# y3
# treated
# (1) terminate, n=24087, avg=15.500481, se=0.029585
# non-treated
# (1) terminate, n=24513, avg=8.486637, se=0.016162
# treated
# (1) terminate, n=24205, avg=15.471615, se=0.029516
# non-treated
# (1) terminate, n=24286, avg=8.499300, se=0.016259
# y4
# treated
# (1) split open_square_2_normalized at 0.573441, n=24087, avg=5.775640, se=0.007329
# 	<=(2) split open_square_0_normalized at 0.241632, n=19790, avg=5.811505, se=0.006773
# 		<=(4) terminate, n=19154, avg=5.813311, se=0.006954
# 		> (5) terminate, n=496, avg=5.747114, se=0.057467
# 	> (3) split open_square_3_normalized at 0.178923, n=4252, avg=5.748469, se=0.032885
# 		<=(6) terminate, n=1697, avg=5.632678, se=0.046818
# 		> (7) terminate, n=2507, avg=5.779855, se=0.039283
# non-treated
# (1) split open_square_0_normalized at 0.200405, n=24513, avg=5.775349, se=0.007285
# 	<=(2) terminate, n=14282, avg=5.745937, se=0.008311
# 	> (3) terminate, n=10231, avg=5.737934, se=0.010649
# treated
# (1) split open_square_2_normalized at 0.573441, n=24205, avg=5.781989, se=0.007274
# 	<=(2) split open_square_0_normalized at 0.241632, n=19752, avg=5.813320, se=0.006712
# 		<=(4) terminate, n=19144, avg=5.816439, se=0.006875
# 		> (5) terminate, n=445, avg=5.793187, se=0.061282
# 	> (3) split open_square_3_normalized at 0.178923, n=4404, avg=5.732764, se=0.028016
# 		<=(6) terminate, n=1778, avg=5.716061, se=0.045659
# 		> (7) terminate, n=2578, avg=5.730950, se=0.033912
# non-treated
# (1) split open_square_0_normalized at 0.200405, n=24286, avg=5.786407, se=0.007277
# 	<=(2) terminate, n=14187, avg=5.757137, se=0.008285
# 	> (3) terminate, n=10099, avg=5.742405, se=0.010677
# y5
# treated
# (1) split open_square_2_normalized at 0.566417, n=24087, avg=10.049320, se=0.013987
# 	<=(2) split bbn_0_normalized at 0.439257, n=19278, avg=10.027728, se=0.013504
# 		<=(4) terminate, n=18832, avg=10.029631, se=0.013858
# 		> (5) terminate, n=307, avg=9.447435, se=0.126809
# 	> (3) split bbn_2_normalized at 0.471940, n=4772, avg=9.262987, se=0.034809
# 		<=(6) terminate, n=2681, avg=9.300367, se=0.046854
# 		> (7) terminate, n=2049, avg=9.039819, se=0.054693
# non-treated
# (1) terminate, n=24513, avg=10.058689, se=0.013851
# treated
# (1) split open_square_2_normalized at 0.566417, n=24205, avg=10.070353, se=0.013946
# 	<=(2) split bbn_0_normalized at 0.439257, n=19217, avg=10.056450, se=0.013539
# 		<=(4) terminate, n=18780, avg=10.054453, se=0.013910
# 		> (5) terminate, n=290, avg=9.466290, se=0.138915
# 	> (3) split bbn_2_normalized at 0.471940, n=4949, avg=9.369464, se=0.041940
# 		<=(6) terminate, n=2759, avg=9.400223, se=0.051049
# 		> (7) terminate, n=2140, avg=9.069469, se=0.052376
# non-treated
# (1) terminate, n=24286, avg=10.073212, se=0.013835


y1
treated
(1) split bbb_2_normalized at 0.743842, n=23908, avg=4.911501, se=0.018753
	<=(2) split bbb_1_normalized at 0.467966, n=7381, avg=1.711621, se=0.016572
		<=(4) terminate, n=5325, avg=1.740061, se=0.027859
		> (5) terminate, n=1990, avg=1.478083, se=0.035216
	> (3) terminate, n=16527, avg=6.291908, se=0.014136
non-treated
(1) terminate, n=24491, avg=1.534381, se=0.007519
treated
(1) split bbb_2_normalized at 0.743842, n=24384, avg=4.940101, se=0.018606
	<=(2) split bbb_1_normalized at 0.467966, n=7435, avg=1.668494, se=0.017332
		<=(4) terminate, n=5376, avg=1.742581, se=0.027869
		> (5) terminate, n=2011, avg=1.444579, se=0.036802
	> (3) terminate, n=16949, avg=6.329640, se=0.013903
non-treated
(1) terminate, n=24308, avg=1.541504, se=0.007417
y2
treated
(1) split open_square_1_normalized at 0.441203, n=23908, avg=10.177663, se=0.023960
	<=(2) split open_square_3_normalized at 0.228590, n=18270, avg=11.049057, se=0.020940
		<=(4) terminate, n=9230, avg=9.759784, se=0.031305


In [ ]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}

input_features = [       
#            'bb_1_normalized', 'bb_0_normalized'
#                 ,
           'bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
           'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized', 
           'open_square_0_normalized', 'open_square_1_normalized', 
           'open_square_2_normalized', 'open_square_3_normalized'
        ]
    
for key in ['assignment']+input_features:
        probabilities_[key] = new_probabilities[key][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3', 'y4', 'y5']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(True, outcome, input_features, 
                                                           min_variance_reduct=-100000.0, max_attempt=30, eps=0.0, delta=0.0
                                                          , min_leaf_size=1000)

    partition.plot_tree(train_result_separate)
    
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    input_features, outcome, eps=0.0)
    
    partition.plot_tree(est_result_separate)
# output
# y1
# /Users/yuany/Library/Python/3.7/bin/ipython:43: RuntimeWarning: invalid value encountered in sqrt
# treated
# (1) split bbb_2_normalized at 0.674970, n=24244, avg=3.806906, se=0.014408
# 	<=(2) split bbb_1_normalized at 0.446568, n=7000, avg=1.526232, se=0.014272
# 		<=(4) split open_square_0_normalized at 0.154120, n=4748, avg=1.537629, se=0.024723
# 			<=(8) terminate, n=3986, avg=1.601708, se=0.026396
# 			> (9) terminate, n=624, avg=1.421517, se=0.052342
# 		> (5) split open_square_3_normalized at 0.249445, n=2191, avg=1.470555, se=0.034126
# 			<=(10) terminate, n=1757, avg=1.514665, se=0.034735
# 			> (11) terminate, n=310, avg=1.427918, se=0.074083
# 	> (3) terminate, n=17244, avg=4.718551, se=0.011916
# non-treated
# (1) split open_square_2_normalized at 0.351767, n=24430, avg=1.540268, se=0.007452
# 	<=(2) terminate, n=15616, avg=1.525120, se=0.007714
# 	> (3) terminate, n=8814, avg=1.575589, se=0.012334
# treated
# (1) split bbb_2_normalized at 0.674970, n=24048, avg=3.826792, se=0.014504
# 	<=(2) split bbb_1_normalized at 0.446568, n=6896, avg=1.515996, se=0.014246
# 		<=(4) split open_square_0_normalized at 0.154120, n=4728, avg=1.506157, se=0.023813
# 			<=(8) terminate, n=3998, avg=1.520046, se=0.026454
# 			> (9) terminate, n=599, avg=1.505572, se=0.051525
# 		> (5) split open_square_3_normalized at 0.249445, n=2109, avg=1.507251, se=0.033432
# 			<=(10) terminate, n=1737, avg=1.487683, se=0.036168
# 			> (11) terminate, n=269, avg=1.581639, se=0.076009
# 	> (3) terminate, n=17152, avg=4.747560, se=0.011906
# non-treated
# (1) split open_square_2_normalized at 0.351767, n=24369, avg=1.536281, se=0.007465
# 	<=(2) terminate, n=15418, avg=1.524415, se=0.007882
# 	> (3) terminate, n=8951, avg=1.555878, se=0.012242
# y2
# treated
# (1) terminate, n=24244, avg=10.160422, se=0.023846
# non-treated
# (1) split bbn_2_normalized at 0.070909, n=24430, avg=4.518390, se=0.013049
# 	<=(2) terminate, n=11280, avg=3.338392, se=0.014661
# 	> (3) terminate, n=13150, avg=5.443780, se=0.014687
# treated
# (1) terminate, n=24048, avg=10.171458, se=0.023843
# non-treated
# (1) split bbn_2_normalized at 0.070909, n=24369, avg=4.533813, se=0.013033
# 	<=(2) terminate, n=11038, avg=3.317100, se=0.014354
# 	> (3) terminate, n=13331, avg=5.455134, se=0.014564
# y3
# treated
# (1) terminate, n=24244, avg=15.480143, se=0.029458
# non-treated
# (1) terminate, n=24430, avg=8.467352, se=0.016277
# treated
# (1) terminate, n=24048, avg=15.495954, se=0.029723
# non-treated
# (1) terminate, n=24369, avg=8.504729, se=0.016214
# y4
# treated
# (1) split open_square_2_normalized at 0.579790, n=24244, avg=5.785440, se=0.007257
# 	<=(2) split open_square_0_normalized at 0.255954, n=20128, avg=5.810806, se=0.006660
# 		<=(4) terminate, n=19590, avg=5.813104, se=0.006830
# 		> (5) terminate, n=390, avg=5.804406, se=0.065294
# 	> (3) split open_square_3_normalized at 0.199352, n=4072, avg=5.828298, se=0.030368
# 		<=(6) terminate, n=1979, avg=5.771693, se=0.044854
# 		> (7) terminate, n=2040, avg=5.767270, se=0.038364
# non-treated
# (1) split open_square_1_normalized at 0.424134, n=24430, avg=5.782859, se=0.007271
# 	<=(2) terminate, n=7201, avg=5.739167, se=0.013508
# 	> (3) terminate, n=17229, avg=5.747370, se=0.007154
# treated
# (1) split open_square_2_normalized at 0.579790, n=24048, avg=5.774059, se=0.007289
# 	<=(2) split open_square_0_normalized at 0.255954, n=20064, avg=5.803747, se=0.006613
# 		<=(4) terminate, n=19563, avg=5.805645, se=0.006770
# 		> (5) terminate, n=343, avg=5.845348, se=0.070489
# 	> (3) split open_square_3_normalized at 0.199352, n=3938, avg=5.742536, se=0.031902
# 		<=(6) terminate, n=1880, avg=5.805632, se=0.045897
# 		> (7) terminate, n=1999, avg=5.787606, se=0.045369
# non-treated
# (1) split open_square_1_normalized at 0.424134, n=24369, avg=5.777597, se=0.007231
# 	<=(2) terminate, n=7257, avg=5.740238, se=0.013337
# 	> (3) terminate, n=17112, avg=5.740034, se=0.007196
# y5
# treated
# (1) terminate, n=24244, avg=10.064657, se=0.014012
# non-treated
# (1) terminate, n=24430, avg=10.046311, se=0.013884
# treated
# (1) terminate, n=24048, avg=10.056521, se=0.014043
# non-treated
# (1) terminate, n=24369, avg=10.065654, se=0.013677


y1


/Users/yuany/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in sqrt


In [ ]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}

input_features = [       
           'bb_1_normalized', 'bb_0_normalized'
            #     ,
            #            'bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
            #            'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized', 
            #            'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
        ]
    
for key in ['assignment']+input_features:
        probabilities_[key] = new_probabilities[key][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3', 'y4', 'y5']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(False, outcome, input_features, 
                                                           min_variance_reduct=0.0, max_attempt=30, 
                                                           eps=0.0, delta=0.0, min_leaf_size=1000)

    partition.plot_tree(train_result_separate)
    
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    input_features, outcome, eps=0.0, separate=False)
    
    partition.plot_tree(est_result_separate)
    
    
# y1
# (1) split bb_1_normalized at 0.344099, n=48532, avg=1.443100, se=0.011784
# 	<=(2) terminate, n=9323, avg=0.107546, se=0.024034
# 	> (3) terminate, n=33691, avg=1.478534, se=0.013961
# (1) split bb_1_normalized at 0.344099, n=48559, avg=1.392826, se=0.011797
# 	<=(2) terminate, n=9424, avg=0.171627, se=0.024664
# 	> (3) terminate, n=33542, avg=1.425723, se=0.014112
# y2
# (1) split bb_1_normalized at 0.388300, n=48532, avg=5.657456, se=0.027147
# 	<=(2) terminate, n=13849, avg=2.553391, se=0.034697
# 	> (3) terminate, n=31158, avg=4.546744, se=0.032548
# (1) split bb_1_normalized at 0.388300, n=48559, avg=5.609517, se=0.027068
# 	<=(2) terminate, n=14067, avg=2.585285, se=0.034111
# 	> (3) terminate, n=30980, avg=4.495645, se=0.032219
# y3
# (1) split bb_0_normalized at 0.685385, n=48532, avg=6.974765, se=0.033709
# 	<=(2) terminate, n=35805, avg=7.041800, se=0.038993
# 	> (3) terminate, n=6008, avg=7.119464, se=0.095559
# (1) split bb_0_normalized at 0.685385, n=48559, avg=7.009412, se=0.033561
# 	<=(2) terminate, n=35601, avg=7.055266, se=0.038882
# 	> (3) terminate, n=6106, avg=6.891756, se=0.091059
# y4
# (1) split bb_0_normalized at 0.652909, n=48532, avg=0.003441, se=0.009640
# 	<=(2) terminate, n=33691, avg=0.032908, se=0.011605
# 	> (3) terminate, n=9323, avg=-0.028139, se=0.021788
# (1) split bb_0_normalized at 0.652909, n=48559, avg=0.002813, se=0.009619
# 	<=(2) terminate, n=33542, avg=0.041880, se=0.011571
# 	> (3) terminate, n=9424, avg=0.067350, se=0.021732
# y5
# (1) split bb_1_normalized at 0.691276, n=48532, avg=-0.002014, se=0.019645
# 	<=(2) terminate, n=35553, avg=-0.029473, se=0.022976
# 	> (3) terminate, n=6072, avg=0.581024, se=0.057550
# (1) split bb_1_normalized at 0.691276, n=48559, avg=0.027404, se=0.019684
# 	<=(2) terminate, n=35850, avg=0.047783, se=0.022898
# 	> (3) terminate, n=5894, avg=0.198482, se=0.058733


In [336]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}

input_features = [       
                       'bb_1_normalized', 'bb_0_normalized',
                       'bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized', 
                       'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
        ]
    
for key in ['assignment']+input_features:
        probabilities_[key] = new_probabilities[key][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3', 'y4', 'y5']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(False, outcome, input_features, 
                                                           min_variance_reduct=0.0, max_attempt=30, eps=0.0, delta=0.0
                                                          , min_leaf_size=1000)

    partition.plot_tree(train_result_separate)
    
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    input_features, outcome, eps=0.0, separate=False)
    
    partition.plot_tree(est_result_separate)
    
# some minimum leaf size constraint does not work for non-separate cases, just need to manually correct it.

y1
(1) split bbb_2_normalized at 0.699611, n=48518, avg=1.425865, se=0.011840
	<=(2) terminate, n=30856, avg=0.015757, se=0.013042
	> (3) terminate, n=4622, avg=1.913454, se=0.035216
(1) split bbb_2_normalized at 0.699611, n=48573, avg=1.428051, se=0.011850
	<=(2) terminate, n=31049, avg=0.018561, se=0.013012
	> (3) terminate, n=4541, avg=2.046389, se=0.034243
y2
(1) split open_square_3_normalized at 0.037864, n=48518, avg=5.647989, se=0.027271
	<=(2) terminate, n=21593, avg=3.106206, se=0.028284
	> (3) terminate, n=26705, avg=4.951016, se=0.032287
(1) split open_square_3_normalized at 0.037864, n=48573, avg=5.627651, se=0.027057
	<=(2) terminate, n=21855, avg=3.062061, se=0.027621
	> (3) terminate, n=26521, avg=4.936906, se=0.032344
y3
(1) split bb_1_normalized at 0.336689, n=48518, avg=7.009515, se=0.033655
	<=(2) split open_square_0_normalized at 0.475251, n=9330, avg=7.209163, se=0.074619
		<=(4) terminate, n=7113, avg=7.089532, se=0.084020
		> (5) terminate, n=313, avg=6.032180, s

In [ ]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}

input_features = [       
#            'bb_1_normalized', 'bb_0_normalized'
            #     ,
                       'bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized', 
                       'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
        ]
    
for key in ['assignment']+input_features:
        probabilities_[key] = new_probabilities[key][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3', 'y4', 'y5']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(False, outcome, input_features, 
                                                           min_variance_reduct=-100000.0, max_attempt=30, eps=0.0, delta=0.0,
                                                           min_leaf_size=1000)

    partition.plot_tree(train_result_separate)
    
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    input_features, outcome, eps=0.0, separate=False)
    
    partition.plot_tree(est_result_separate)

## ignore below

In [292]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}

input_features = [       
                       'bb_1_normalized', 'bb_0_normalized',
                       'bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized', 
                       'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
        ]
    
for key in ['assignment']+input_features:
        probabilities_[key] = new_probabilities[key][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3', 'y4', 'y5']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(False, outcome, input_features, 
                                                           min_variance_reduct=-100000.0, max_attempt=30, eps=0.0, delta=0.0)

    partition.plot_tree(train_result_separate)
    
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    input_features, outcome, eps=0.0, separate=False)
    
    partition.plot_tree(est_result_separate)

y1
(1) split bbb_2_normalized at 0.696285, n=48613, avg=2.277747, se=0.016260
	<=(2) split bb_0_normalized at 0.645159, n=31011, avg=0.000693, se=0.012900
		<=(4) terminate, n=16041, avg=0.009142, se=0.017982
		> (5) terminate, n=9314, avg=-0.223639, se=0.023630
	> (3) terminate, n=4628, avg=1.474200, se=0.043274
separate
(1) split bbb_2_normalized at 0.696285, n=48478, avg=2.269474, se=0.016175
	<=(2) split bb_0_normalized at 0.645159, n=30894, avg=0.002394, se=0.013008
		<=(4) terminate, n=16132, avg=-0.004417, se=0.018062
		> (5) terminate, n=9032, avg=-0.069225, se=0.023257
	> (3) terminate, n=4535, avg=1.667818, se=0.044798
y2
(1) terminate, n=48613, avg=5.636932, se=0.026898
separate
(1) terminate, n=48478, avg=5.643788, se=0.027315
y3
(1) terminate, n=48613, avg=6.982218, se=0.033433
separate
(1) terminate, n=48478, avg=7.003959, se=0.033823
y4
(1) split bb_1_normalized at 0.356949, n=48613, avg=-0.001976, se=0.010298
	<=(2) terminate, n=9519, avg=-0.014680, se=0.023542
	> (3) s

In [293]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}

input_features = [       
#            'bb_1_normalized', 'bb_0_normalized'
            #     ,
                       'bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized', 
                       'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
        ]
    
for key in ['assignment']+input_features:
        probabilities_[key] = new_probabilities[key][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3', 'y4', 'y5']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(False, outcome, input_features, 
                                                           min_variance_reduct=-100000.0, max_attempt=30, eps=0.0, delta=0.0)

    partition.plot_tree(train_result_separate)
    
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    input_features, outcome, eps=0.0, separate=False)
    
    partition.plot_tree(est_result_separate)
    
# y1
# (1) split bbb_1_normalized at 0.192227, n=48603, avg=1.422314, se=0.011774
# 	<=(2) terminate, n=32944, avg=1.943577, se=0.012794
# 	> (3) split open_square_3_normalized at 0.027006, n=15634, avg=0.251349, se=0.019595
# 		<=(6) terminate, n=4975, avg=0.159776, se=0.034427
# 		> (7) terminate, n=10284, avg=0.263745, se=0.024381
# (1) split bbb_1_normalized at 0.192227, n=48488, avg=1.413600, se=0.011798
# 	<=(2) terminate, n=32862, avg=1.945815, se=0.012771
# 	> (3) split open_square_3_normalized at 0.027006, n=15596, avg=0.283886, se=0.019513
# 		<=(6) terminate, n=4922, avg=0.173399, se=0.034463
# 		> (7) terminate, n=10338, avg=0.314793, se=0.024165
# y2
# (1) split open_square_3_normalized at 0.055976, n=48603, avg=5.668103, se=0.027074
# 	<=(2) terminate, n=24963, avg=3.286808, se=0.027226
# 	> (3) terminate, n=23432, avg=5.027140, se=0.034366
# (1) split open_square_3_normalized at 0.055976, n=48488, avg=5.598890, se=0.027088
# 	<=(2) terminate, n=24780, avg=3.259264, se=0.027336
# 	> (3) terminate, n=23504, avg=4.875091, se=0.034148
# y3
# (1) split open_square_1_normalized at 0.552957, n=48603, avg=7.035132, se=0.033658
# 	<=(2) terminate, n=39997, avg=7.054099, se=0.036996
# 	> (3) terminate, n=8315, avg=6.246653, se=0.066663
# (1) split open_square_1_normalized at 0.552957, n=48488, avg=6.948759, se=0.033546
# 	<=(2) terminate, n=39870, avg=6.952568, se=0.036806
# 	> (3) terminate, n=8305, avg=6.160022, se=0.063780
# y4
# (1) split bbn_0_normalized at 0.443783, n=48603, avg=0.006838, se=0.009665
# 	<=(2) terminate, n=38675, avg=0.039409, se=0.010836
# 	> (3) split bbb_1_normalized at 0.345544, n=5092, avg=0.099604, se=0.030046
# 		<=(6) terminate, n=2597, avg=-0.000533, se=0.042038
# 		> (7) terminate, n=273, avg=0.249042, se=0.124117
# (1) split bbn_0_normalized at 0.443783, n=48488, avg=-0.000595, se=0.009594
# 	<=(2) terminate, n=38617, avg=0.041593, se=0.010753
# 	> (3) split bbb_1_normalized at 0.345544, n=4989, avg=-0.022320, se=0.030603
# 		<=(6) terminate, n=2645, avg=-0.084055, se=0.043066
# 		> (7) terminate, n=272, avg=-0.050528, se=0.121091
# y5
# (1) split bbn_0_normalized at 0.440097, n=48603, avg=0.045040, se=0.019611
# 	<=(2) terminate, n=38554, avg=0.051438, se=0.022025
# 	> (3) split open_square_1_normalized at 0.558101, n=5270, avg=-0.241064, se=0.058324
# 		<=(6) terminate, n=2145, avg=0.040893, se=0.096819
# 		> (7) terminate, n=1205, avg=-0.192442, se=0.101052
# (1) split bbn_0_normalized at 0.440097, n=48488, avg=-0.019645, se=0.019717
# 	<=(2) terminate, n=38532, avg=-0.036041, se=0.022076
# 	> (3) split open_square_1_normalized at 0.558101, n=5114, avg=-0.419234, se=0.064614
# 		<=(6) terminate, n=2144, avg=-0.110425, se=0.105630
# 		> (7) terminate, n=1177, avg=-0.586002, se=0.107941


y1
(1) split bbb_2_normalized at 0.700891, n=48580, avg=2.273520, se=0.016177
	<=(2) split bbn_0_normalized at 0.423766, n=30974, avg=0.003344, se=0.013004
		<=(4) split bbn_2_normalized at 0.176051, n=20256, avg=0.007678, se=0.016057
			<=(8) terminate, n=9882, avg=-0.006833, se=0.023310
			> (9) terminate, n=9961, avg=0.026548, se=0.022710
		> (5) terminate, n=6042, avg=-0.196956, se=0.030027
	> (3) split open_square_3_normalized at 0.223369, n=4521, avg=1.475666, se=0.043717
		<=(6) terminate, n=2600, avg=2.701541, se=0.049266
		> (7) terminate, n=1078, avg=-0.102390, se=0.066174
separate
(1) split bbb_2_normalized at 0.700891, n=48511, avg=2.273705, se=0.016259
	<=(2) split bbn_0_normalized at 0.423766, n=31063, avg=-0.000361, se=0.012861
		<=(4) split bbn_2_normalized at 0.176051, n=20438, avg=0.007540, se=0.015924
			<=(8) terminate, n=10137, avg=-0.024397, se=0.022910
			> (9) terminate, n=9894, avg=0.028695, se=0.022981
		> (5) terminate, n=5988, avg=-0.138282, se=0.028682
	> (

In [ ]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}
for key in ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
            'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized'
#             ,
#             'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
           ]:
    for a in [0, 1]:
        probabilities_[(a, key)] = new_probabilities[(a, key)][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(True, outcome, ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                                                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized'
#                                                                            ,
#                                                     'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
                                                                          ], 
                                                           min_variance_reduct=0, max_attempt=20, eps=0.0, delta=0.0)

    partition.plot_tree(train_result_separate)
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                     'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized'
#                                      ,
#                                     'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
                                    ], outcome, eps=0.0)
    
    partition.plot_tree(est_result_separate)

In [130]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}
for key in ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
            'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized',
            'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
           ]:
    for a in [0, 1]:
        probabilities_[(a, key)] = new_probabilities[(a, key)][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3']:
    # create a new partition algo
    print(outcome)

    train_result_separate = partition.split_exposure_hajek(True, outcome, ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                                                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized',
                                                    'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'], 
                                                           min_variance_reduct=0, max_attempt=10, eps=0.0, delta=0.0)

    partition.plot_tree(train_result_separate)
    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                     'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized',
                                    'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
                                    ], outcome, eps=0.0)
    
    partition.plot_tree(est_result_separate)

y1
treated
(1) split bbn_2_normalized at 0.427377, n=24128, avg=3.815668, se=0.014441
	<=(2) terminate, n=14107, avg=4.170775, se=0.010414
	> (3) terminate, n=10021, avg=3.279713, se=0.012440
non-treated
(1) split open_square_1_normalized at 0.585793, n=24593, avg=1.546005, se=0.007412
	<=(2) terminate, n=20662, avg=1.557561, se=0.002290
	> (3) terminate, n=3886, avg=1.498137, se=0.029796
treated
(1) split bbn_2_normalized at 0.427377, n=24164, avg=3.805915, se=0.014446
	<=(2) terminate, n=14126, avg=4.166535, se=0.010372
	> (3) terminate, n=10038, avg=3.275434, se=0.012980
non-treated
(1) split open_square_1_normalized at 0.585793, n=24206, avg=1.543980, se=0.007475
	<=(2) terminate, n=20515, avg=1.564246, se=0.002281
	> (3) terminate, n=3663, avg=1.452190, se=0.037044
y2
treated
(1) split open_square_2_normalized at 0.586785, n=24128, avg=10.156672, se=0.023821
	<=(2) terminate, n=20424, avg=10.352721, se=0.007511
	> (3) terminate, n=3664, avg=9.254257, se=0.104954
non-treated
(1) sp

In [125]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}
for key in ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
            'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized',
            'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
           ]:
    for a in [0, 1]:
        probabilities_[(a, key)] = new_probabilities[(a, key)][idx]

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3']:
    # create a new partition algo
    train_result_separate = partition.split_exposure_hajek(True, outcome, ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                                                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized',
                                                    'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'], 
                                                           min_variance_reduct=0, max_attempt=10, eps=0.0, delta=0.0)

    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                     'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized',
                                    'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
                                    ], outcome, eps=0.0)
    print(outcome)
    partition.plot_tree(train_result_separate)
    partition.plot_tree(est_result_separate)
    
    
    
# pure cutoff
data['y1'] = data['neighbor'] * 0.1 + data['gender'] * 1 + \
            (data['bbb_2_normalized'] > 0.7).astype(float) * 2 + \
            data['assignment'] * (data['bbb_2_normalized'] > 0.7).astype(float) * (data['bbn_2_normalized'] < 0.4).astype(float) * 3 + \
            np.random.normal(0, 1, len(data))

# structural diversity is causal
data['y2'] = \
    data['neighbor'] * 0.1 + data['gender'] * 1 + \
    data['structural_diversity'] + \
    data['assignment'] * data['structural_diversity'] * 1 + \
    np.random.normal(0, 1, len(data))

# structural diversity is causal
data['y3'] = \
    data['neighbor'] * 0.1 + data['gender'] * 1 + \
    data['structural_diversity_1'] + \
    data['assignment'] * data['structural_diversity_1'] * 1 + \
    np.random.normal(0, 1, len(data))


# make naive tree fail

y1
(1) split open_square_3_normalized at 0.234978, n=24090, avg=3.811211, se=0.014459
	<=(2) split bbb_2_normalized at 0.786622, n=15177, avg=4.142351, se=0.009216
		<=(4) terminate, n=6819, avg=2.093329, se=0.013768
		> (5) terminate, n=8358, avg=5.871601, se=0.018293
	> (3) terminate, n=8913, avg=3.229865, se=0.013591

(1) split open_square_1_normalized at 0.645940, n=24407, avg=1.554045, se=0.007434
	<=(2) terminate, n=23155, avg=1.557846, se=0.001224
	> (3) terminate, n=1229, avg=1.398752, se=0.054313
(1) split open_square_3_normalized at 0.234978, n=24202, avg=3.810367, se=0.014428
	<=(2) split bbb_2_normalized at 0.786622, n=15212, avg=4.128285, se=0.009220
		<=(4) terminate, n=6666, avg=2.030811, se=0.013837
		> (5) terminate, n=8546, avg=5.841779, se=0.018132
	> (3) terminate, n=8990, avg=3.252613, se=0.013725

(1) split open_square_1_normalized at 0.645940, n=24392, avg=1.535951, se=0.007452
	<=(2) terminate, n=23085, avg=1.540942, se=0.001224
	> (3) terminate, n=1272, avg=1.3

In [126]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}
for key in ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
            'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized',
            'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
           ]:
    for a in [0, 1]:
        probabilities_[(a, key)] = new_probabilities[(a, key)][idx]
    
partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3']:
    # create a new partition algo
    train_result_separate = partition.split_exposure_hajek(False, outcome, ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                                                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized',
                                                    'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'], 
                                                           min_variance_reduct=0, max_attempt=10, eps=0., delta=0.005)

    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                     'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized',
                                    'open_square_0_normalized', 'open_square_1_normalized', 'open_square_2_normalized', 'open_square_3_normalized'
                                    ], outcome, eps=0., separate=False)
    print(outcome)
    partition.plot_tree(train_result_separate)
    partition.plot_tree(est_result_separate)

31147.452914108086
162521.84433213843
42435.518069831116
173860.090993481
33508.883533358414
86608.30675860087
47196.08521920537
93186.54632298491
33533.883533358414
86608.30675860087
47244.50774693787
93186.54632298491
31138.452914108082
162521.84433213843
42414.60021729613
173860.090993481
27128.95688945216
552363.2218799482
33617.70046075612
585793.8139761603
30957.391683242735
162855.1776654718
42015.69701320572
174189.68956396877
21938.32410281949
344616.41771000903
23633.002207471967
393423.3471968782
29037.26509335432
316108.5043387984
37844.17179071189
335871.617345175
27797.762359085646
439238.8819165061
35129.56918236898
466860.1477973245
24663.89432197082
633140.1254243891
28236.304574757665
680035.7678391207
40444.21565553318
116653.25948967024
60820.05590027159
174909.2173721313
38349.32459347173
111844.03569473207
57640.66741173468
167817.38992618636
41742.009449137375
120530.10896209422
62754.21229546117
181001.2611899438
38828.429710795746
112942.36977034333
58380.25175

KeyboardInterrupt: 

In [93]:
partition.plot_tree(train_result_separate)

(1) split bbb_2_normalized at 0.703332, n=48610, avg=2.268135, se=0.016185
	<=(2) split bbb_0_normalized at 0.750755, n=31038, avg=0.016423, se=0.013726
		<=(4) terminate, n=14575, avg=0.020952, se=0.018571
		> (5) split open_square_1_normalized at 0.494922, n=3165, avg=0.000668, se=0.018571
			<=(10) terminate, n=1293, avg=-0.083426, se=0.061573
			> (11) terminate, n=1453, avg=-0.020097, se=0.061573
	> (3) split open_square_3_normalized at 0.164934, n=4656, avg=1.433390, se=0.013726
		<=(6) terminate, n=2220, avg=2.801616, se=0.048394
		> (7) terminate, n=1605, avg=0.407212, se=0.048394


In [135]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}
for key in ['bb_0_normalized', 'bb_1_normalized']:
    probabilities_[key] = new_probabilities[key][idx]
    
partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y1', 'y2', 'y3']:
    # create a new partition algo
    train_result_separate = partition.split_exposure_hajek(True, outcome, ['bb_0_normalized', 'bb_1_normalized'], 
                                                           min_variance_reduct=0, max_attempt=40, eps=0.0, delta=0.0)

    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    ['bb_0_normalized', 'bb_1_normalized'], outcome)
    print(outcome)
    partition.plot_tree(train_result_separate)
    partition.plot_tree(est_result_separate)

y1
(1) split bb_1_normalized at 0.672682, n=22703, avg=3.658353, se=0.014186
	<=(2) terminate, n=10756, avg=3.891183, se=0.013753
	> (3) terminate, n=11947, avg=3.387671, se=0.011127

(1) terminate, n=22568, avg=1.550628, se=0.007828
(1) split bb_1_normalized at 0.672682, n=22542, avg=3.652991, se=0.014152
	<=(2) terminate, n=10786, avg=3.871400, se=0.013583
	> (3) terminate, n=11756, avg=3.390191, se=0.011381

(1) terminate, n=22544, avg=1.574090, se=0.007824
y2
(1) terminate, n=22703, avg=9.916405, se=0.024090

(1) split bb_0_normalized at 0.713566, n=22568, avg=4.489116, se=0.013507
	<=(2) terminate, n=12203, avg=5.512292, se=0.007774
	> (3) terminate, n=10365, avg=3.333035, se=0.014921
(1) terminate, n=22542, avg=9.909139, se=0.023875

(1) split bb_0_normalized at 0.713566, n=22544, avg=4.468808, se=0.013529
	<=(2) terminate, n=12123, avg=5.490585, se=0.007900
	> (3) terminate, n=10421, avg=3.337115, se=0.015156
y3
(1) terminate, n=22703, avg=15.118571, se=0.029850

(1) terminate, 

(1) split bbn_1_normalized at 0.499883, n=22583, avg=10.736642, se=0.041584
	<=(2) terminate, n=10988, avg=14.015357, se=0.047284
	> (3) terminate, n=11595, avg=7.610136, se=0.033021

(1) terminate, n=22669, avg=-1.911704, se=0.013539


In [64]:
partition.plot_tree(train_result_separate)

(1) split bbn_1_normalized at 0.499883, n=22583, avg=10.736642, se=0.041584
	<=(2) terminate, n=10988, avg=14.015357, se=0.047284
	> (3) terminate, n=11595, avg=7.610136, se=0.033021

(1) terminate, n=22669, avg=-1.911704, se=0.013539


In [92]:
partition.plot_tree(est_result_separate)

(1) terminate, n=25057, avg=4.802173, se=0.015294

(1) terminate, n=25115, avg=2.991581, se=0.013223


In [68]:
data['y'] = data['neighbor'] * 0.1 + \
    data['assignment'] * (data['bbb_2_normalized'] >= 0.5) + \
    data['assignment'] * (data['bbb_2_normalized'] >= 0.8) * data['bbb_2_normalized'] + \
    np.random.normal(0, 1, len(data))

/var/svcscm/.bento/kernels/bento_kernel_default/latest/bento_kernel_default:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [67]:
np.mean(data[data['bbn_0_normalized'] <= 0.055]['bbn_2_normalized'] > 0.5)


0.49614600203946974

In [150]:
# create a new partition algo
data_ = data[idx]

probabilities_ = {}
for key in ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
            'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized']:
    probabilities_[key] = new_probabilities[key][idx]

    


/var/svcscm/.bento/kernels/bento_kernel_default/latest/bento_kernel_default:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [151]:

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y4']:
    # create a new partition algo
    train_result_separate = partition.split_exposure_hajek(True, outcome, ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                                                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized'], 
                                                           min_variance_reduct=0, max_attempt=40, eps=0.005, delta=0.005)

    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                     'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized'], outcome, eps=0.005)
    print(outcome)
    partition.plot_tree(train_result_separate)
    partition.plot_tree(est_result_separate)



y4
(1) terminate, n=22553, avg=5.626320, se=0.007685

(1) terminate, n=22612, avg=5.619031, se=0.007641
(1) terminate, n=22692, avg=5.625559, se=0.007684

(1) terminate, n=22500, avg=5.604758, se=0.007697


In [152]:

partition = causalPartition(data_, probabilities_, 'assignment')

for outcome in ['y4']:
    # create a new partition algo
    train_result_separate = partition.split_exposure_hajek(True, outcome, ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                                                       'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized'], 
                                                           min_variance_reduct=0, max_attempt=40, eps=0.005, delta=0.005, naive=True)

    # estimate on a separate set
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                     'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized'], outcome, eps=0.005)
    print(outcome)
    partition.plot_tree(train_result_separate)
    partition.plot_tree(est_result_separate)

y4
(1) split bbb_0_normalized at 0.108669, n=22691, avg=5.632678, se=0.007684
	<=(2) terminate, n=19103, avg=5.662145, se=0.006504
	> (3) terminate, n=3588, avg=5.475790, se=0.013115

(1) split bbb_0_normalized at 0.086782, n=22730, avg=5.611182, se=0.007639
	<=(2) terminate, n=1747, avg=5.789750, se=0.018950
	> (3) terminate, n=20983, avg=5.596315, se=0.005665
(1) split bbb_0_normalized at 0.108669, n=22554, avg=5.619158, se=0.007684
	<=(2) terminate, n=18878, avg=5.643668, se=0.006504
	> (3) terminate, n=3676, avg=5.525561, se=0.013004

(1) split bbb_0_normalized at 0.086782, n=22382, avg=5.612654, se=0.007699
	<=(2) terminate, n=1749, avg=5.827437, se=0.018915
	> (3) terminate, n=20633, avg=5.592738, se=0.005690


In [146]:
    est_result_separate = partition.estimate_exposure_hajek(train_result_separate, 
                                    ['bbb_0_normalized', 'bbb_1_normalized', 'bbb_2_normalized', 
                                     'bbn_0_normalized', 'bbn_1_normalized', 'bbn_2_normalized'], outcome, eps=0.005)


In [147]:
est_result_separate

{1: {'feature': 'bbb_0_normalized',
  'threshold': 0.2519524806243262,
  'left_result': {'hajek': 2.556272757539806,
   'hajek_se': 0.0055302901356506,
   'mse': 46553.88638500526,
   'N': 20258},
  'right_result': {'hajek': 2.441951818090727,
   'hajek_se': 0.015617613033011775,
   'mse': 4789.291518539594,
   'N': 2240},
  'hajek': 2.551676255591129,
  'hajek_se': 0.007085797948989498,
  'mse': 25411.2919155181,
  'N': 22498},
 0: {'feature': 'bbb_0_normalized',
  'threshold': 0.02390246037821797,
  'left_result': {'hajek': 2.6722433011939986,
   'hajek_se': 0.017557083445665422,
   'mse': 3646.5453111356996,
   'N': 1749},
  'right_result': {'hajek': 2.5425230431540715,
   'hajek_se': 0.005198469551441739,
   'mse': 43969.74619833013,
   'N': 20719},
  'hajek': 2.5455021733751786,
  'hajek_se': 0.007104477591679239,
  'mse': 25477.362603907088,
  'N': 22468}}

In [138]:
probabilities['bbb_2_normalized']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


/var/svcscm/.bento/kernels/bento_kernel_default/latest/bento_kernel_default:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
np.corrcoef(np.array(data['y1']), np.array(data['structural_diversity']))

array([[1.        , 0.27195546],
       [0.27195546, 1.        ]])